In [1]:
#conv Neural Network
# tensorboard --logdir=/home/ncc/notebook/learn/tensorboard/log
"""
created by kim Seong jung

RESNET MODULE 2016/2/7
CONVOLUTION MODULE 2016/2/6
batch_normalization 적용
3/14일 model save and restore 적용
n_train=300 이 코드 수정하기 
def Batch_random 에 valideation and test validate 하는 함수에 오류를 일부로 구현-->나중에 고치기 

수정사항

def divide_images(images , labels,batch_size):
    ...
    ...
    ...
    image = images[ -batch_size :  ] 
    label = labels[ -batch_size :  ]
    list_images.append(image)
    list_labels.append(label)

    return list_images,list_labels
    2017_4/7 ver 3로 바꿈



수정사항 
tf.cond을 만들어서 처리한다.
"""
import sys
import numpy as np 
import tensorflow as tf
import re
import random
import math
import time
import matplotlib.pyplot as plt
%matplotlib inline
import os 
from multiprocessing import Pool

# Load Image 

In [2]:
def load_images_cls(path_img_list , label_num ):
    n_paths = len(path_img_list)
    cls=np.zeros([n_paths])
    cls=np.full_like(cls, label_num)
    list_imgs=[]
    for i , path in  enumerate(path_img_list):
        msg = "\r progress {0} /{1}".format(str(i),n_paths)
        sys.stdout.write(msg)
        sys.stdout.flush()
        
        img=np.load(path)
        list_imgs.append(img)
    images=np.asarray(list_imgs)

    return images , cls 

# Load Image Using multiProc

In [3]:
def load_img(path):
    img=np.load(path)
    return img
def load_images_cls_multiproc(path_img_list , label_num ):
    n_paths = len(path_img_list)
    cls=np.zeros([n_paths])
    cls=np.full_like(cls, label_num)
    list_imgs=[]
    pool=Pool()
    count=0
    for img in  pool.imap( load_img,path_img_list):
        msg = "\r progress {0} /{1}".format(str(count),n_paths)
        sys.stdout.write(msg)
        sys.stdout.flush()
        list_imgs.append(img)
        count+=1
    images=np.asarray(list_imgs)

    return images , cls 

# load Training , Validation , Test 

In [4]:
list_=[1,2,3]
list_2=[3,4,5]
type(list_) is list

True

In [5]:
file_locate ='/ssd/n_vs_ab/type1/1/' #Folder data saved#WB_eye_15_PRE_A_B_C_incepAx3/
ori_img_row = 300
ori_img_col = 300
in_ch =3
n_classes=3
img_size_cropped=288
batch_size=30

In [6]:
def crawl_folder(folder_path): #search_str 찾고자 하는 string 
    search_path_list=[]
    fileList = os.walk(folder_path).next()[2]
    subFolder_list = os.walk(folder_path).next()[1]
    if(len(fileList)!=0):
        for j in range(len(fileList)):
            search_path_list.append(folder_path+'/'+fileList[j])
    if len(subFolder_list)==0:
        return search_path_list
    else: 
        for i in range(len(subFolder_list)):
            search_path_list.extend(crawl_folder(folder_path+'/'+subFolder_list[i] ))
        return search_path_list

In [7]:
ret_path=crawl_folder('/ssd/ret/')
normal_path_1=crawl_folder('/ssd/normal_1/')
"""
normal_path_2=crawl_folder('/ssd/normal_2/')
"""

"\nnormal_path_2=crawl_folder('/ssd/normal_2/')\n"

# 7 classification

# 2 Classification

In [8]:
ret_imgs , ret_cls = load_images_cls(ret_path , 0)
#cat_gka_imgs , cat_gka_cls = load_images_cls(cat_gka_path , 1)
#ret_imgs , ret_cls = load_images_cls(ret_path , 1)
#ret_cat_imgs , ret_cat_cls = load_images_cls( ret_cat_path , 3)
#ret_glu_imgs , ret_glu_cls = load_images_cls(ret_glu_path , 4)
#gla_imgs , gla_cls = load_images_cls(gla_path , 2)
normal_1_imgs , normal_1_cls = load_images_cls_multiproc(normal_path_1[:7000] , 1)


 progress 6999 /7000

In [9]:
print np.shape(ret_imgs)
#print np.shape(ret_imgs)
#print np.shape(gla_imgs)
print np.shape(normal_1_imgs)

(3359, 300, 300, 3)
(7000, 300, 300, 3)


In [10]:
def cls2onehot(n_classes,list_cls):
    tmp_list=[]
    size=len(list_cls)
    ret_onehot=np.zeros([size,n_classes])
    for i , cls in enumerate(list_cls):
        cls=int(cls)
        ret_onehot[i,cls]=1
    return ret_onehot

In [11]:
ret_labs=cls2onehot(n_classes, ret_cls)
#ret_labs=cls2onehot(n_classes , ret_cls)
#gla_labs = cls2onehot(n_classes , gla_cls)
normal_1_labs = cls2onehot(n_classes , normal_1_cls)

# define Variable , and placeholder 

In [12]:
x_= tf.placeholder("float",shape=[batch_size,ori_img_row , ori_img_col , in_ch],  name = 'x-input')
y_= tf.placeholder("float",shape=[batch_size , n_classes] , name = 'y-input')
y_cls = tf.argmax(y_ , axis = 1)

keep_prob = tf.placeholder("float")
#x_image= tf.reshape(x_,[-1,ori_img_row,ori_img_col,in_ch])
phase_train= tf.placeholder(tf.bool , name='phase_train')
bn_flag = tf.placeholder(tf.bool , name='bn_flag')
bn_decay = tf.placeholder("float")

place_info={}
place_info['x_']=x_
place_info['y_']=y_
place_info['keep_prob']=keep_prob
place_info['phase_train']=phase_train

# Image ProPrecessing 

In [13]:
# This function takes a single image as input,
# and a boolean whether to build the training or testing graph.
def pre_processing(image ):
    def training(image):
        # For training, add the following to the TensorFlow graph.

        # Randomly crop the input image.
        image = tf.random_crop(image, size=[img_size_cropped, img_size_cropped, in_ch])

        # Randomly flip the image horizontally.
        image = tf.image.random_flip_left_right(image)
        image = tf.image.random_flip_up_down(image)
        # Randomly adjust hue, contrast and saturation.
        image = tf.image.random_hue(image, max_delta=0.05)
        image = tf.image.random_contrast(image, lower=0.3, upper=1.0)
        image = tf.image.random_brightness(image, max_delta=0.2)
        image = tf.image.random_saturation(image, lower=0.0, upper=2.0)

        # Some of these functions may overflow and result in pixel
        # values beyond the [0, 1] range. It is unclear from the
        # documentation of TensorFlow 0.10.0rc0 whether this is
        # intended. A simple solution is to limit the range.

        # Limit the image pixels between [0, 1] in case of overflow.
        #image = tf.minimum(image, 1.0)
        #image = tf.maximum(image, 0.0)
        return image
    def testing(image):
        # For training, add the following to the TensorFlow graph.

        # Crop the input image around the centre so it is the same
        # size as images that are randomly cropped during training.
        image = tf.image.resize_image_with_crop_or_pad(image,target_height=img_size_cropped,target_width=img_size_cropped)
        return image
    image=tf.cond( phase_train , lambda : training(image) \
                  , lambda : testing(image) ,name='pre_processing')
    return image

In [14]:
def pre_process(images, training):
    # Use TensorFlow to loop over all the input images and call
    # the function above which takes a single image as input.
    images = tf.map_fn(lambda image: pre_process_image(image, training), images)

    return images

In [15]:
def PRE_PROCESSING(images, phase_train ,img_size_cropped  , device):
    print 
    # Use TensorFlow to loop over all the input images and call
    # the function above which takes a single image as input.
    images = tf.map_fn( lambda image: pre_process_image(image, phase_train ,img_size_cropped  , device), images)
    print 'distorted image\'s shape is' , images.get_shape()
    return images


In [16]:

"""def weight_variable(name,shape):
    #initial = tf.truncated_normal(shape , stddev=0.1)
    initial = tf.get_variable(name,shape=shape , initializer = tf.contrib.layers.xavier_initializer())
    return tf.Variable(initial)"""
def bias_variable(shape , name):
    initial = tf.constant(0.1 , shape=shape , )
    return tf.Variable(initial , name=name)

def conv2d(x,w,strides_ , name):
    return tf.nn.conv2d(x,w, strides = strides_, padding='SAME' , name = name)

def max_pool(x , ksize , strides , padding , name ):
    return tf.nn.max_pool(x ,ksize , strides , padding ,name=name)
def make_weights_biases(layer_name
                        , w_name , b_name, ksize ,device_name , initializer='xavier',\
                        restore_flag =False , restore_path = './WB_save'):    
    """
    Doc
    the folder include weights and bises that will be restored is located in './WB_save'
    if you want to change save path ,  change restore_path  
    """
    if len(ksize)==4: # convolution filter shape [batch , row , col , color_ch]
        out_ch=ksize[3]
    elif len(ksize)==2: #fully connected layer shape [in_ch , output_ch]
        out_ch=ksize[1]
    with tf.device(device_name):
        with tf.variable_scope(layer_name) as scope:
            if restore_flag==False:
                print layer_name+' make weigths and biases'

                try:
                    w_conv = tf.get_variable(w_name, ksize , initializer = tf.contrib.layers.xavier_initializer())
                except:
                    scope.reuse_variables()
                    w_conv = tf.get_variable(w_name, ksize , initializer = tf.contrib.layers.xavier_initializer())        
                try:
                    b_conv = bias_variable([out_ch] , name=b_name)
                except:
                    scope.reuse_variables()
                    b_conv = bias_variable([out_ch],name=b_name)
                return w_conv , b_conv
            elif restore_flag == True:
                print layer_name+' load weigths and biases'
                try:
                    file_list=crawl_folder(restore_path)
                    for path in file_list:
                        name=path.split('/')[-1].split('.')[0] #Bottlenect_b_W1:0
                        if layer_name+'_'+w_name == name:
                            print layer_name+'_'+w_name+'was restored!!'
                            w_conv = tf.Variable(np.load(path) , name = w_name)
                except:
                    scope.reuse_variables()
                    file_list=crawl_folder(restore_path)
                    for path in file_list:
                        name=path.split('/')[-1].split('.')[0] #Bottlenect_b_W1
                        if layer_name+'_'+w_name == name:
                            print layer_name+'_'+w_name+'was restored'
                            w_conv = tf.Variable(np.load(path) , name = w_name)         
                try:
                    file_list=crawl_folder(restore_path)
                    for path in file_list:
                        name=path.split('/')[-1].split('.')[0] #Bottlenect_b_W1
                        if layer_name+'_'+b_name == name:
                            print layer_name+'_'+b_name+'was restored!!'
                            b_conv = tf.Variable(np.load(path),name = b_name)        
                except:
                    file_list=crawl_folder(restore_path)
                    for path in file_list:
                        name=path.split('/')[-1].split('.')[0] #Bottlenect_b_W1
                        if layer_name+'_'+b_name == name:
                            print layer_name+'_'+b_name+'was restored'                            
                            b_conv = tf.Variable(np.load(path),name = b_name)        
                return w_conv , b_conv 

In [17]:
def make_conv_info( x,w_conv,b_conv,c_strides,c_pooling ):
    """
    
    conv_info['x']=x
    conv_info['w_conv']=w_conv
    conv_info['c_strides']=c_strides
    conv_info['c_pooling']=c_pooling
    conv_info['name']=name
    conv_info['b_conv']=b_conv
    
    """
    conv_info={}
    
    conv_info['x']=x
    conv_info['w_conv']=w_conv
    conv_info['c_strides']=c_strides
    conv_info['c_pooling']=c_pooling
    conv_info['b_conv']=b_conv
    
    return conv_info

In [18]:
def bn(layer  , layer_name  ,  decay , restore_flag=False , restore_path ='./WB_save/' ):
    #hyper parameter : decay 
    #
    n_out = layer.get_shape()[3] 
    if restore_flag == False:
        beta = tf.Variable(tf.constant(0.0, shape=[n_out]),
                                     name='beta', trainable=True)
        gamma = tf.Variable(tf.constant(1.0, shape=[n_out]),
                                      name='gamma', trainable=True)
    elif restore_flag == True:
        beta = tf.Variable(np.load(restore_path+layer_name+'_beta.npy') , name='beta')
        gamma = tf.Variable(np.load(restore_path+layer_name+'_gamma.npy') , name ='gamma')
        print layer_name+'beta was restored'
        print layer_name+'gamma was restored'
        
    batch_mean, batch_var = tf.nn.moments(layer, [0,1,2], name='moments')
    ema = tf.train.ExponentialMovingAverage( decay , name="EMA")
    ema_apply_op = ema.apply([batch_mean, batch_var])
    ema_mean=ema.average(batch_mean) ;ema_var=ema.average(batch_var) 
    layer_BN = tf.nn.batch_normalization(layer, batch_mean, batch_var, beta, gamma, 1e-3 , name = 'BN')    
    layer_relu=tf.nn.relu(layer_BN ,name='bn_relu')
    return layer_relu

In [19]:
def convolution(conv_info , place_info , bn_flag , layer_name , restore_flag , restore_path):
    """
    place_info['x_']=x_
    place_info['y_']=y_
    place_info['keep_prob']=keep_prob
    place_info['train_flag'] =train_flag
    place_info['bn_flag'] =bn_flag
    place_info['bn_decay'] = bn_decay
    conv_info['x']=x
    conv_info['w_conv']=w_conv
    conv_info['c_strides']=c_strides
    conv_info['c_pooling']=c_pooling
    conv_info['name']=name
    conv_info['b_conv']=b_conv    
    """
    with tf.variable_scope(layer_name) as scope:
        layer = tf.nn.conv2d(conv_info['x'] , conv_info['w_conv'] , conv_info['c_strides'] ,\
                             conv_info['c_pooling'] ,name='conv')+conv_info['b_conv']
        if bn_flag==True:
            layer_relu=bn(layer , layer_name , 0.999 , restore_flag , restore_path)
        if bn_flag==False:
            layer_relu=tf.nn.relu(layer ,name='no_bn_relu')
        layer_drop = tf.nn.dropout(layer_relu,place_info['keep_prob'] , name='dropout')
        return layer_drop        

# RESNET

In [20]:
def PRE_LAYER(x , device_,layer_name,n_nodes , place_info  , bn_flag  , restore_flag , restore_path):
    """
    place_info['x_']=x_
    place_info['y_']=y_
    place_info['keep_prob']=keep_prob
    place_info['train_flag'] =train_flag
    place_info['bn_flag'] = train_flag
    place_info['bn_decay'] = bn_decay
    conv
    """
    in_ch = x.get_shape()[3]
    out_ch=n_nodes
    c_ksize=[7,7 , in_ch, out_ch]
    c_pooling ='SAME'
    c_strides=[1,2,2,1] # when downsampling featrue map , set strides parameter,[1,2,2,1]
    w_conv , b_conv =make_weights_biases(layer_name,'W' ,'B' , c_ksize ,device_ ,'xavier',restore_flag ,  restore_path )
    conv_info=make_conv_info(x, w_conv, b_conv, c_strides, c_pooling)
    layer1=convolution(conv_info , place_info , bn_flag , layer_name+'_L' , restore_flag , restore_path)   
    p_ksize=[1,3,3,1]
    p_strides=[1,2,2,1]
    with tf.variable_scope(layer_name+'_end') as scope:
        layer2 = tf.nn.max_pool(layer1 ,p_ksize, c_strides , c_pooling , name='maxpool')
    print layer2.get_shape()
    return layer2

In [21]:
def RES_2(x , device_,layer_name,n_nodes , place_info  , bn_flag  , restore_flag , restore_path):
    """
    out_ch1 , out_ch2 = n_nodes
    out_ch1 , out_ch2=n_nodes
    conv_info['x']=x
    conv_info['w_conv']=w_conv
    conv_info['c_strides']=c_strides
    conv_info['c_pooling']=c_pooling
    conv_info['name']=name
    conv_info['b_conv']=b_conv
    문제는 형이 변할때인데 이미지가 축소할때 기본의  x는 이미지를 살려서 패치를 해야 한다,
    그것을 텐서 플로어를 이용해서 해야 한다는 것이다.    
    """

    
    """만약 입력한 영상의 row와 또는 col이 layer2의 row 또는 col과 다르다면 padding 을 시켜야 한다.
    넘어갈때는 폴링을 시키는것이 아니라 1x1 conv 에 1,2,2,1 스트라이드를 시키는지는 정확히 모르겠다
    또 논문에 보면 차원이 늘어날때 extra padding 이라고 하는데 단순히 zeropadding을 뒤에 붙이는건지는
    잘 모르겠다. 어짜피 뒤에 있는 레이어와 더해져 문제 될것 같지는 않은데 잘 모르겠다.
    """

    #pad_row=int((x.get_shape()[1]-layer2.get_shape()[1])/2)
    #pad_col=int((x.get_shape()[2]-layer2.get_shape()[2])/2)

    """add padding 
    아직 구현 미완료
    x_padded=tf.pad(x,paddings=[[pad_col,pad_col],[pad_row,pad_row]],mode="CONSTANT",name=layer_name+'x_padded')
    x_padded=tf.nn.max_pool(x,[1,2,2,1],[1,2,2,1],padding='SAME' , name=layer_name+'x_padded')
    x_ch=int(x_padded.get_shape()[3])
    print x_padded.get_shape()

    """

    #여기에 batch  가 들어가야 한다.

    """
    여기에만 Batch Normalization 을 추가하는 게 좋을까 
    아니면 다른곳에도 batch normalization 을 추가하는게 좋을까?
    그리고 이것을 어떻게 수식으로 표현할수 있을까?

    """

    in_ch = x.get_shape()[3]

    out_ch1,out_ch2=n_nodes
    out_ch_shortcut = out_ch2
    
    c_ksize1=[3,3 , in_ch   , out_ch1]
    c_ksize2=[3,3 , out_ch1 , out_ch2]
    c_ksize_shortcut = [1,1,in_ch,out_ch2]
    
    c_strides1=strides # when downsampling featrue map , set strides parameter,[1,2,2,1]
    c_strides2=[1,1,1,1]
    c_strides_shortcut=strides
    
    
    c_pooling1='SAME'
    c_pooling2='SAME'
    c_pooling_shortcut="SAME"
    
    
    w_conv1 , b_conv1 =make_weights_biases(layer_name+'_1', 'W' ,'B' ,c_ksize1 ,device_ ,'xavier',restore_flag , restore_path )
    w_conv2 , b_conv2= make_weights_biases(layer_name+'_2', 'W' ,'B' , c_ksize2 ,device_ ,'xavier',restore_flag , restore_path)
    w_conv_shortcut , b_conv_shortcut =make_weights_biases(layer_name+'shortcut', 'W' ,'B' , c_ksize_shortcut ,device_ ,'xavier',restore_flag , restore_path)
    
    conv_info_1=make_conv_info(x, w_conv1, b_conv1, c_strides1, c_pooling1)    
    layer1=convolution(conv_info_1 , place_info , bn_flag ,layer_name+'_L1' , restore_flag , restore_path)   

    conv_info_2=make_conv_info(layer1, w_conv2, b_conv2, c_strides2, c_pooling2)
    layer2=convolution(conv_info_2 , place_info , bn_flag, layer_name+'_L2' , restore_flag , restore_path)

    conv_info_shortcut = make_conv_info(x, w_conv_shortcut, b_conv_shortcut, c_strides_shortcut, c_pooling_shortcut)
    layer_shortcut=convolution(conv_info_shortcut , place_info , bn_flag , layer_name+'Lshortcut' , restore_flag , restore_path)

    with tf.variable_scope(layer_name+'_end') as scope:
        layer_sum=tf.add(layer2 ,layer_shortcut , name="add")    
        layer_sum_relu=tf.nn.relu(layer_sum , name=layer_name+"relu")    
    print layer_sum_relu.get_shape()

    return layer_sum_relu



In [22]:
def BOTTLENECT(x , device_,layer_name,n_nodes , strides , bn_flag , restore_flag  , restore_path ):
    """
    usage:
    n_nodes=(n_nodes[0],n_nodes[1],n_nodes[2])
    out_ch1 , out_ch2 , out_ch3 = n_nodes
    """
    """만약 입력한 영상의 row와 또는 col이 layer2의 row 또는 col과 다르다면 padding 을 시켜야 한다.
    넘어갈때는 폴링을 시키는것이 아니라 1x1 conv 에 1,2,2,1 스트라이드를 시키는지는 정확히 모르겠다
    또 논문에 보면 차원이 늘어날때 extra padding 이라고 하는데 단순히 zeropadding을 뒤에 붙이는건지는
    잘 모르겠다. 어짜피 뒤에 있는 레이어와 더해져 문제 될것 같지는 않은데 잘 모르겠다.
    """

    #pad_row=int((x.get_shape()[1]-layer2.get_shape()[1])/2)
    #pad_col=int((x.get_shape()[2]-layer2.get_shape()[2])/2)

    """add padding 
    아직 구현 미완료
    x_padded=tf.pad(x,paddings=[[pad_col,pad_col],[pad_row,pad_row]],mode="CONSTANT",name=layer_name+'x_padded')
    x_padded=tf.nn.max_pool(x,[1,2,2,1],[1,2,2,1],padding='SAME' , name=layer_name+'x_padded')
    x_ch=int(x_padded.get_shape()[3])
    print x_padded.get_shape()

    """
    ############################# layer4 #############################

    #여기에 batch  가 들어가야 한다.

    """
    여기에만 Batch Normalization 을 추가하는 게 좋을까 
    아니면 다른곳에도 batch normalization 을 추가하는게 좋을까?
    그리고 이것을 어떻게 수식으로 표현할수 있을까?

    """

    #############################layer1#############################
    in_ch = x.get_shape()[3]
    
    out_ch1,out_ch2,out_ch3=n_nodes
    c_ksize1=[1,1 , in_ch   , out_ch1]
    c_ksize2=[3,3 , out_ch1 , out_ch2]
    c_ksize3=[1,1 , out_ch2 , out_ch3]
    c_ksize_shortcut=[1,1,in_ch,out_ch3]
    
    c_pooling1="SAME";c_strides1=strides;
    c_pooling2="SAME";c_strides2=[1,1,1,1];
    c_pooling3="SAME";c_strides3=[1,1,1,1];
    c_pooling_shortcut="SAME";c_strides_shortcut=strides;
    
    
    w_conv1 , b_conv1 =make_weights_biases(layer_name+'_1', 'W' ,'B' ,c_ksize1 ,device_ ,'xavier',restore_flag , restore_path  )
    w_conv2 , b_conv2= make_weights_biases(layer_name+'_2', 'W' ,'B' ,c_ksize2 ,device_ ,'xavier',restore_flag , restore_path)
    w_conv3 , b_conv3= make_weights_biases(layer_name+'_3', 'W' ,'B' ,c_ksize3 ,device_ ,'xavier',restore_flag , restore_path)
    w_conv_shortcut , b_conv_shortcut =make_weights_biases(layer_name+'_shortcut', 'W' ,'B' ,c_ksize_shortcut,device_,'xavier',restore_flag,restore_path)
    
    
    conv_info_1=make_conv_info(x, w_conv1, b_conv1, c_strides1, c_pooling1) 
    layer1=convolution(conv_info_1 , place_info , bn_flag , layer_name+'_L1' ,restore_flag , restore_path)
    
    conv_info_2=make_conv_info(layer1, w_conv2, b_conv2, c_strides2, c_pooling2)
    layer2=convolution(conv_info_2, place_info , bn_flag , layer_name+'_L2',restore_flag , restore_path)
    
    conv_info_3=make_conv_info(layer2, w_conv3, b_conv3, c_strides3, c_pooling3)
    layer3=convolution(conv_info_3, place_info , bn_flag , layer_name+'_L3',restore_flag , restore_path)
    
    conv_info_shortcut = make_conv_info(x, w_conv_shortcut, b_conv_shortcut, c_strides_shortcut, c_pooling_shortcut)
    layer_shortcut=convolution(conv_info_shortcut , place_info , bn_flag , layer_name+'_Lshortcut',restore_flag , restore_path)

    with tf.variable_scope(layer_name+'_end') as scope: 
        layer_sum=tf.add(layer3 ,layer_shortcut , name="sum")
    print layer_sum.get_shape()
    return layer_sum

            


# INCEPTION_NET_V4

In [23]:
def STEM_A(x , device_,layer_name,n_nodes , place_info  , bn_flag  , restore_flag , restore_path):
    """ 
        standard = 32;32;64;96
        out_ch1 , out_ch2 , out_ch3 , out_ch4 =n_nodes   
        out_ch1=n_nodes[0]
        c_ksize1=[1,1 , in_ch   , out_ch1]
        c_strides1=strides;c_pooling1="SAME"
        w_conv1 , b_conv1 =make_weights_biases\
        (layer_name+'_1', 'W' ,'B' ,c_ksize1 ,device_ ,'xavier',restore_flag , restore_path  )
        conv_info_1=make_conv_info(x, w_conv1, b_conv1, c_strides1, c_pooling1)
        layer1=convolution(conv_info_1 , place_info , bn_flag , layer_name+'_1_bn' ,restore_flag , restore_path)

    """    
    with tf.device(device_):
        out_ch1=32;out_ch2=32;out_ch3=64;out_ch4 =96
        in_ch=x.get_shape()[3]
        
        ################################## layer 1 ##################################  
        c_ksize1=[3,3,in_ch , out_ch1]
        c_strides1=[1,2,2,1]
        c_pooling1='VALID'
        w_conv1 , b_conv1 =make_weights_biases(layer_name+'_1' , 'W' ,'B', c_ksize1 , device_,'xavier',restore_flag,  restore_path)
        conv_info_1=make_conv_info(x, w_conv1, b_conv1, c_strides1, c_pooling1)
        layer1=convolution(conv_info_1 , place_info , bn_flag , layer_name+'_L1' ,restore_flag , restore_path)        
        ################################## layer 2 ##################################  
        c_ksize2=[3,3,out_ch1,out_ch2]
        w_conv2 , b_conv2= make_weights_biases(layer_name+'_2' , 'W' ,'B', c_ksize2 , device_,'xavier',restore_flag,  restore_path)
        c_strides2=[1,1,1,1];c_pooling2='VALID'
        conv_info_2=make_conv_info(layer1, w_conv2, b_conv2, c_strides2, c_pooling2)
        layer2=convolution(conv_info_2 , place_info , bn_flag , layer_name+'_L2' ,restore_flag , restore_path)
        ################################## layer 3 ##################################  
        c_ksize3=[3,3,out_ch2 , out_ch3]
        w_conv3 , b_conv3= make_weights_biases(layer_name+'_3' , 'W' ,'B' ,c_ksize3 , device_,'xavier',restore_flag,  restore_path)
        c_strides3=[1,1,1,1]
        c_pooling3='SAME'
        conv_info_3=make_conv_info(layer2, w_conv3, b_conv3, c_strides3, c_pooling3)
        layer3=convolution(conv_info_3 , place_info , bn_flag , layer_name+'_L3' ,restore_flag , restore_path)
        ################################## layer 4 ##################################  
        c_ksize4=[3,3,out_ch3 , out_ch4]
        w_conv4 , b_conv4= make_weights_biases(layer_name+'_4' , 'W' ,'B', c_ksize4 , device_,'xavier',restore_flag,  restore_path)
        c_strides4=[1,2,2,1]
        c_pooling4='VALID'
        conv_info_4=make_conv_info(layer3, w_conv4, b_conv4, c_strides4, c_pooling4)
        layer4=convolution(conv_info_4 , place_info , bn_flag , layer_name+'_L4' ,restore_flag , restore_path)
        ################################## layer 4 brach ##################################          
        b_p_ksize4=[1,3,3,1]
        b_p_strides4=[1,2,2,1]
        b_p_padding4 ='VALID'
        with tf.variable_scope('b_'+layer_name+'_4') as scope:
            b_layer4=tf.nn.max_pool(layer3 , b_p_ksize4, b_p_strides4, b_p_padding4 ,name='max_pool') #b is branch
            b_layer4_relu=tf.nn.relu(b_layer4 , name = 'relu')
        with tf.variable_scope(layer_name+'_end') as scope:
            concat_layer=tf.concat(3 , [layer4 , b_layer4_relu] , name='CONCAT')
        return concat_layer

In [24]:
def STEM_B(x,device_,layer_name , place_info  , bn_flag  , restore_flag , restore_path):
    """
    stardard =>64,64,64,96 
    b_out_ch1,b_out_ch2=64, 96
    """
    with tf.device(device_):
        in_ch=x.get_shape()[3]
        #################################################################################
        out_ch1=64;out_ch2=64;out_ch3=64;out_ch4=96; 
        ##############################right side layer 1##################################
        c_ksize1=[1,1,in_ch,out_ch1]
        w_conv1 , b_conv1=make_weights_biases(layer_name+'_1' , 'W' ,'B', c_ksize1 ,device_,'xavier',restore_flag,  restore_path)
        c_strides1=[1,1,1,1]
        c_pooling1='SAME'
        conv_info_1=make_conv_info(x, w_conv1, b_conv1, c_strides1, c_pooling1)
        layer1=convolution(conv_info_1 , place_info , bn_flag , layer_name+'_L1' ,restore_flag , restore_path)
        ##############################right side layer 2##################################
        c_ksize2=[7,1,out_ch1 , out_ch2]
        w_conv2 , b_conv2= make_weights_biases(layer_name+'_2' , 'W' ,'B',c_ksize2 ,device_,'xavier',restore_flag,  restore_path)
        c_strides2=[1,1,1,1]
        c_pooling2='SAME'
        conv_info_2=make_conv_info(layer1, w_conv2, b_conv2, c_strides2, c_pooling2)
        layer2=convolution(conv_info_2 , place_info , bn_flag , layer_name+'_L2' ,restore_flag , restore_path)
        ##############################right side layer 3##################+################
        c_ksize3=[1,7,out_ch2 , out_ch3]
        w_conv3 , b_conv3= make_weights_biases(layer_name+'_3' , 'W' ,'B',c_ksize3 ,device_,'xavier',restore_flag,  restore_path)
        c_strides3=[1,1,1,1]
        c_pooling3='SAME'
        conv_info_3=make_conv_info(layer2, w_conv3, b_conv3, c_strides3, c_pooling3)
        layer3=convolution(conv_info_3 , place_info , bn_flag , layer_name+'_L3' ,restore_flag , restore_path)
        ##############################right side layer 4##################################
        c_ksize4=[3,3,out_ch3 , out_ch4]
        w_conv4 , b_conv4= make_weights_biases(layer_name+'_4' , 'W' ,'B', c_ksize4 , device_ ,'xavier',restore_flag,restore_path)
        c_strides4=[1,1,1,1]
        c_pooling4='VALID'
        conv_info_4=make_conv_info(layer3, w_conv4, b_conv4, c_strides4, c_pooling4)
        layer4=convolution(conv_info_4 , place_info , bn_flag , layer_name+'_L4' ,restore_flag , restore_path)
        ##############################left side##################################
        b_out_ch1=64;b_out_ch2=96
        ##############################layer_1_branch####################################
        
        b_c_ksize1=[1,1,in_ch , b_out_ch1]
        b_w_conv1 , b_b_conv1 =make_weights_biases('b_'+layer_name+'_1', 'W' , 'B', b_c_ksize1 ,device_,'xavier',restore_flag,restore_path)
        b_c_strides1=[1,1,1,1]
        b_c_pooling1='SAME'
        b_conv_info1=make_conv_info(x, b_w_conv1, b_b_conv1, b_c_strides1, b_c_pooling1)
        b_layer_1=convolution(b_conv_info1 , place_info , bn_flag , 'b_'+layer_name+'_L1' ,restore_flag , restore_path)
        ##############################layer_2_branch####################################'b_'+
        
        b_c_ksize2=[3,3,b_out_ch1 , b_out_ch2]
        b_w_conv2 , b_b_conv2= make_weights_biases('b_'+layer_name+'_2' , 'W' , 'B',b_c_ksize2 ,device_,'xavier',restore_flag,  restore_path)
        b_c_strides2=[1,1,1,1]
        b_c_pooling2='VALID'
        b_conv_info_2=make_conv_info(b_layer_1, b_w_conv2, b_b_conv2, b_c_strides2, b_c_pooling2)
        b_layer_2=convolution(b_conv_info_2 , place_info , bn_flag , 'b_'+layer_name+'_L2' ,restore_flag , restore_path)        
        ##############################concatenate layers###########################
        with tf.variable_scope(layer_name+'end') as scope:
            concat_layer=tf.concat(3 , [layer4 , b_layer_2] , name="CONCAT")
    return concat_layer

In [25]:
def STEM_C( x , device_,layer_name , place_info  , bn_flag  , restore_flag , restore_path):
    """
    out_ch = n_nodes
    """
    with tf.device(device_):
        in_ch=x.get_shape()[3]
        #########################################################################
        out_ch = 192
        c_ksize=[3,3,in_ch,out_ch]
        w_conv_1 , b_conv_1 =make_weights_biases(layer_name+'_1' , 'W' , 'B',c_ksize , device_,'xavier',restore_flag,  restore_path)
        c_strides=[1,1,1,1]
        c_pooling='SAME'
        conv_info=make_conv_info(x,w_conv_1 , b_conv_1, c_strides, c_pooling)
        layer=convolution(conv_info , place_info , bn_flag , layer_name+'_L1' ,restore_flag , restore_path)

        #########################################################################
        
        b_p_ksize=[1,2,2,1]
        b_p_strides=[1,1,1,1]
        b_p_pooling='SAME'
        with tf.variable_scope(layer_name+'_L2') as scope:
            b_layer = tf.nn.max_pool(layer , b_p_ksize , b_p_strides , b_p_pooling,name='max_pool' )
            b_layer_relu = tf.nn.relu(b_layer,name='relu')
        #########################################################################
        with tf.variable_scope(layer_name+'end') as scope:        
            concat_layer=tf.concat(3,[layer , b_layer_relu] , name = 'concat')
    return concat_layer

In [26]:
def INCEPTION_MODULE_A(x , device_,layer_name, place_info  , bn_flag  , restore_flag , restore_path):

    """
    out_ch1,out_ch2=n_nodes
    input X shape is [n_batch , row , col, out_ch]
    35 x 35 grid
    """
    #######################################layer1##########################################
    in_ch=x.get_shape()[3]
    
    out_ch1=64 ;c_strides1=[1,1,1,1];c_ksize1=[1,1,in_ch,out_ch1];c_pooling1='SAME'
    out_ch2=96 ;c_strides2=[1,1,1,1];c_ksize2=[3,3,out_ch1,out_ch2];c_pooling2='SAME'
             
    w_conv1 , b_conv1 = make_weights_biases(layer_name+'_1', 'W', 'B',c_ksize1,device_,'xavier',restore_flag,restore_path)
    w_conv2 , b_conv2 = make_weights_biases(layer_name+'_2' ,'W' ,'B',c_ksize2,device_,'xavier',restore_flag,restore_path)
    
    conv_info_1=make_conv_info(x, w_conv1, b_conv1, c_strides1, c_pooling1)
    layer1=convolution(conv_info_1 , place_info , bn_flag , layer_name+'_L1',restore_flag , restore_path)
    conv_info_2=make_conv_info(layer1, w_conv2, b_conv2, c_strides2, c_pooling2)
    layer2=convolution(conv_info_2 , place_info , bn_flag , layer_name+'_L2',restore_flag , restore_path)
    ####################################layer_1_branch#############################################                     

    b1_p_ksize=[1,2,2,1];b1_p_strides=[1,1,1,1];b1_p_pooling='SAME'
    with tf.variable_scope('b1_'+layer_name+'_1') as scope:
        b1_layer1      =tf.nn.avg_pool(x, b1_p_ksize , b1_p_strides,b1_p_pooling ,name='avg_pool')
        b1_layer1_relu =tf.nn.relu(b1_layer1,name='relu')
    b1_out_ch=96
    b1_c_ksize=[1,1, in_ch , b1_out_ch]
    b1_c_strides=[1,1,1,1]
    b1_c_pooling ='SAME'
    b1_w_conv,b1_b_conv=make_weights_biases('b1_'+layer_name,'W','B',b1_c_ksize , device_ ,'xavier',restore_flag,  restore_path)                
    b1_conv_info=make_conv_info(b1_layer1_relu,b1_w_conv,b1_b_conv,b1_c_strides, b1_c_pooling)
    b1_layer=convolution(b1_conv_info , place_info , bn_flag , 'b1_'+layer_name+'_L' ,restore_flag,  restore_path)

    ####################################layer_2_branch#############################################                     

    b2_out_ch=96;b2_c_ksize=[1,1,in_ch,b2_out_ch];b2_c_strides=[1,1,1,1];b2_c_pooling='SAME'
    b2_w_conv , b2_b_conv=make_weights_biases('b2_'+layer_name,'W','B',b2_c_ksize , device_ ,'xavier',restore_flag, restore_path)
    b2_conv_info=make_conv_info(x, b2_w_conv, b2_b_conv, b2_c_strides, b2_c_pooling)
    b2_layer=convolution(b2_conv_info , place_info , bn_flag , 'b2_'+layer_name+'_L',restore_flag,  restore_path)
    ####################################layer_3_branch#############################################                     

    b3_out_ch1=64;b3_c_ksize1 =[1,1,     in_ch , b3_out_ch1];b3_c_strides1=[1,1,1,1];b3_c_pooling1='SAME'
    b3_out_ch2=96;b3_c_ksize2 =[3,3,b3_out_ch1 , b3_out_ch2];b3_c_strides2=[1,1,1,1];b3_c_pooling2='SAME'
    b3_out_ch3=96;b3_c_ksize3 =[3,3,b3_out_ch2 , b3_out_ch3];b3_c_strides3=[1,1,1,1];b3_c_pooling3='SAME'
    
    b3_w_conv1 , b3_b_conv1 = make_weights_biases('b3_'+layer_name+'_1','W','B',b3_c_ksize1 , device_ ,'xavier',restore_flag,  restore_path)
    b3_w_conv2 , b3_b_conv2 = make_weights_biases('b3_'+layer_name+'_2','W','B',b3_c_ksize2 , device_ ,'xavier',restore_flag,  restore_path)
    b3_w_conv3 , b3_b_conv3 = make_weights_biases('b3_'+layer_name+'_3','W','B',b3_c_ksize3 , device_ ,'xavier',restore_flag,  restore_path)
    
    b3_conv_info1=make_conv_info(x, b3_w_conv1, b3_b_conv1, b3_c_strides1, b3_c_pooling1)
    b3_layer_1=convolution(b3_conv_info1 , place_info , bn_flag ,'b3_'+layer_name+'_L1' ,restore_flag,  restore_path)
    b3_conv_info2=make_conv_info(b3_layer_1, b3_w_conv2, b3_b_conv2, b3_c_strides2, b3_c_pooling2)    
    b3_layer_2=convolution(b3_conv_info2 , place_info , bn_flag ,'b3_'+layer_name+'_L2' ,restore_flag, restore_path)
    b3_conv_info3=make_conv_info(b3_layer_2, b3_w_conv3, b3_b_conv3, b3_c_strides3, b3_c_pooling3)
    b3_layer_3=convolution(b3_conv_info3 , place_info , bn_flag , 'b3_'+layer_name+'_L3' ,restore_flag,  restore_path)

    #################################################################################                     
    with tf.variable_scope(layer_name+'_end') as scope:
        layer_concat=tf.concat(3,[layer2,b1_layer ,b2_layer,b3_layer_3 ],name = 'concat')
        print layer_concat
    return layer_concat

In [27]:
def INCEPTION_MODULE_B(x , device_,layer_name,n_nodes , place_info  , bn_flag  , restore_flag , restore_path):

    """
    for 17 X 17 grid 
    """
    in_ch =x.get_shape()[3]
    out_ch1 =192 ; out_ch2=224; out_ch3 =256
    c_ksize1 = [1,1, in_ch  ,out_ch1]
    c_ksize2 = [1,7, out_ch1,out_ch2]
    c_ksize3 = [7,1, out_ch2,out_ch3]

    w_conv1 , b_conv1 =make_weights_biases( layer_name+'_1','W' ,'B', c_ksize1 , device)
    w_conv2 , b_conv2 =make_weights_biases( layer_name+'_2','W' ,'B', c_ksize2 , device)
    w_conv3 , b_conv3 =make_weights_biases( layer_name+'_3','W' ,'B', c_ksize3 , device)
    
    c_strides1 =[1,1,1,1]
    c_strides2 =[1,1,1,1]
    c_strides3 =[1,1,1,1]
    
    c_pooling1 ='SAME'
    c_pooling2 ='SAME'
    c_pooling3 ='SAME'

    conv_info1=make_conv_info(x, w_conv1, b_conv1, c_strides1, c_pooling1)
    layer_1=convolution(conv_info_1 , place_info , bn_flag ,layer_name+'_L1' ,'xavier',restore_flag,  restore_path)
    conv_info2=make_conv_info(layer_1, w_conv2, b_conv2, c_strides2, c_pooling2)
    layer_2=convolution(conv_info2 , place_info , bn_flag ,layer_name+'_L2' ,'xavier',restore_flag,  restore_path)
    conv_info3=make_conv_info(x, w_conv3, b_conv3, c_strides3, c_pooling3)
    layer_3=convolution(conv_info3 , place_info , bn_flag ,layer_name+'_L3' ,'xavier',restore_flag,  restore_path)


    #####################################layer_1_branch############################################ 
    b1_p_ksize=[1,2,2,1]
    b1_p_strides=[1,1,1,1]
    b1_p_pooling ='SAME'
    with tf.variable_scope(layer_name+'b1_layer1') as scope:
        b1_layer1      =tf.nn.avg_pool(x , b1_p_ksize , b1_p_strides , b1_p_pooling ,name='conv')
        b1_layer1_relu =tf.nn.relu(b1_layer1,name='relu')

    b1_out_ch2 = 128
    b1_c_ksize2=[1,1,in_ch,b1_out_ch]
    b1_w_conv2 , b1_b_conv2 = make_weights_biases('b1_'+layer_name,'W','B',b1_c_ksize, device )
    b1_c_strides2=[1,1,1,1]
    b1_c_pooling2='SAME'
    b1_conv_info2=make_conv_info(b1_layer1_relu , b1_w_conv2, b1_b_conv2, b1_c_strides2, b1_c_pooling2)
    b1_layer2=convolution(b1_conv_info2 , place_info , bn_flag ,layer_name+'_L2' ,'xavier',restore_flag,  restore_path)

    #####################################layer_2_branch############################################ 
    b2_out_ch=384
    b2_c_ksize=[1,1,in_ch,b2_out_ch]
    b2_w_conv , b2_b_conv =make_weights_biases('b2_'+layer_name,'W','B',b2_c_ksize , device)
    b2_c_strides=[1,1,1,1]
    b2_c_pooling='SAME'
    b2_conv_info=make_conv_info(x , b2_w_conv, b1_b_conv, b2_c_strides, b2_c_pooling)
    b2_layer=convolution(b2_conv_info , place_info , bn_flag ,layer_name+'_L' ,'xavier',restore_flag,  restore_path)
    #####################################layer_3_branch############################################ 


    b3_out_ch1=192; b3_out_ch2 =192; b3_out_ch3=224 ; b3_out_ch4=224 ; b3_out_ch5 =256
    b3_c_ksize1=[1,1,in_ch,b3_out_ch1]
    b3_c_ksize2=[1,7,b3_out_ch1 , b3_out_ch2]
    b3_c_ksize3=[7,1,b3_out_ch2 , b3_out_ch3]
    b3_c_ksize4=[1,7,b3_out_ch3 , b3_out_ch4]
    b3_c_ksize5=[7,1,b3_out_ch4 , b3_out_ch5]

    b3_w_conv1 , b3_b_conv1=make_weights_biases('b3_'+layer_name+'_1','W','B',b3_c_ksize1, device)
    b3_w_conv2 , b3_b_conv2=make_weights_biases('b3_'+layer_name+'_2','W','B',b3_c_ksize2, device)
    b3_w_conv3 , b3_b_conv3=make_weights_biases('b3_'+layer_name+'_3','W','B',b3_c_ksize3, device)
    b3_w_conv4 , b3_b_conv4=make_weights_biases('b3_'+layer_name+'_4','W','B',b3_c_ksize4, device)
    b3_w_conv5 , b3_b_conv5=make_weights_biases('b3_'+layer_name+'_5','W','B',b3_c_ksize5, device)
    
    b3_c_strides1=[1,1,1,1]
    b3_c_strides2=[1,1,1,1]
    b3_c_strides3=[1,1,1,1]
    b3_c_strides4=[1,1,1,1]
    b3_c_strides5=[1,1,1,1]
    
    b3_c_pooling1="SAME"
    b3_c_pooling2="SAME"
    b3_c_pooling3="SAME"
    b3_c_pooling4="SAME"
    b3_c_pooling5="SAME"
    
    b3_conv_info1=make_conv_info(x , b3_w_conv1, b3_b_conv1, b3_c_strides1, b3_c_pooling1)
    b3_layer1=convolution(b3_conv_info1 , place_info , bn_flag ,'b3_'+layer_name+'_L1' ,'xavier',restore_flag,  restore_path)
    
    b3_conv_info2=make_conv_info(b3_layer1 , b3_w_conv2, b3_b_conv2, b3_c_strides2, b2_c_pooling)
    b3_layer2=convolution(b3_conv_info2 , place_info , bn_flag ,'b3_'+layer_name+'_L2' ,'xavier',restore_flag,  restore_path)
    
    b3_conv_info3=make_conv_info(b3_layer2 , b3_w_conv3, b3_b_conv3, b3_c_strides3, b2_c_pooling)
    b3_layer3=convolution(b3_conv_info3 , place_info , bn_flag ,'b3_'+layer_name+'_L3' ,'xavier',restore_flag,  restore_path)
    
    b3_conv_info4=make_conv_info(b3_layer3 , b3_w_conv4, b3_b_conv4, b3_c_strides4, b2_c_pooling)
    b3_layer4=convolution(b3_conv_info4 , place_info , bn_flag ,'b3_'+layer_name+'_L4' ,'xavier',restore_flag,  restore_path)
    
    b3_conv_info5=make_conv_info(b3_layer4 , b3_w_conv5, b3_b_conv5, b3_c_strides5, b2_c_pooling)
    b3_layer5=convolution(b3_conv_info5 , place_info , bn_flag ,'b3_'+layer_name+'_L5' ,'xavier',restore_flag,  restore_path)
   
    ################################################################################# 

    with tf.variable_scope(layer_name+'_end') as scope:
        layer_concat=tf.concat(3, [layer_3 , b1_layer2 ,b2_layer ,b3_layer5] ,name='concat')
    return layer_concat


In [28]:
def INCEPTION_MODULE_C(x , device_,layer_name,n_nodes , place_info  , bn_flag  , restore_flag , restore_path):

    """
    for 8 X 8 grid modules
    """
    print x.get_shape()
    in_ch=x.get_shape()[3]
    
    out_ch1 = 384 ; out_ch2_a =256 ;out_ch2_b = 256
    c_ksize1 = [1,1,in_ch , out_ch1]
    c_ksize2_a = [1,3,out_ch1 , out_ch2_a]
    c_ksize2_b = [3,1,out_ch1 , out_ch2_b]
    w_conv1 ,b_conv1=make_weights_biases(layer_name+'_1','W','B', c_ksize1 ,device)
    w_conv2_a ,b_conv2_a=make_weights_biases(layer_name+'_2_a','W','B', c_ksize2_a ,device)
    w_conv2_b ,b_conv2_b=make_weights_biases(layer_name+'_2_b','_W_b','_B_b', c_ksize2_b ,device)
    c_strides1=[1,1,1,1]
    c_strides2_a=[1,1,1,1]
    c_strides2_b=[1,1,1,1]
    c_pooling1='SAME'
    c_pooling2_a='SAME'
    c_pooling2_b='SAME'
    conv_info1=make_conv_info(x, w_conv1, b_conv1, c_strides1, c_pooling1)
    layer1=convolution(conv_info_1 , place_info , bn_flag ,layer_name+'_L1' ,'xavier',restore_flag,  restore_path)
    conv_info2_a=make_conv_info(layer_1, w_conv2_a, b_conv2_a, c_strides2_a, c_pooling2_a)
    layer2_a=convolution(conv_info2_a , place_info , bn_flag ,layer_name+'_L2_a' ,'xavier',restore_flag,  restore_path)
    conv_info2_b=make_conv_info(layer_1, w_conv2_b, b_conv2_b, c_strides2_b, c_pooling2_b)
    layer2_b=convolution(conv_info2_b , place_info , bn_flag ,layer_name+'_L2_b' ,'xavier',restore_flag,  restore_path)
    ################################################################################# 


    b1_p_ksize1=[1,2,2,1]
    b1_p_strides1=[1,1,1,1]
    b1_p_pooling1='SAME'
    with tf.variable_scope('b1_'+layer_name+'_1') as scope:    
        b1_layer1     = tf.nn.avg_pool(x , b1_p_ksize1 , b1_p_strides1 , b1_p_pooling1 , name='avg_pool')
        b1_layer1_relu= tf.nn.relu(b1_layer1 , name='relu')

    b1_out_ch2 =256
    b1_c_ksize2=[1,1,in_ch , b1_out_ch]
    b1_w_conv2 , b1_b_conv2= make_weights_biases('b1_'+layer_name+'_2', 'W' ,'B', b1_c_ksize , device)
    b1_c_pooling2 = 'SAME';b1_c_strides = [1,1,1,1];
    b1_conv_info2=make_conv_info(x, b1_w_conv2, b1_b_conv2, b1_c_strides, b1_c_pooling2)
    b1_layer2=convolution(b1_conv_info2 , place_info , bn_flag ,'b1_'+layer_name+'_L2' ,'xavier',restore_flag,  restore_path)
    ################################################################################# 
    b2_out_ch=256
    b2_c_ksize=[1,1,in_ch,b2_out_ch]
    b2_w_conv , b2_b_conv= make_weights_biases('b2_'+layer_name , 'W' , 'B',b2_c_ksize , device)
    b2_c_pooling = 'SAME';b2_c_strides=[1,1,1,1];
    b2_conv_info=make_conv_info(x, b1_w_conv2, b1_b_conv2, b1_c_strides, b1_c_pooling2)
    b2_layer2=convolution(b2_conv_info , place_info , bn_flag ,'b2_'+layer_name+'_L2' ,'xavier',restore_flag,  restore_path)
   
    ################################################################################# 


    b3_out_ch1=384;b3_out_ch2=448;b3_out_ch3=512;b3_out_ch4_a=256;b3_out_ch4_b=256
    b3_c_ksize1 =[1,1,in_ch , b3_out_ch1]
    b3_c_ksize2 =[1,3,b3_out_ch1 , b3_out_ch2]
    b3_c_ksize3 =[3,1,b3_out_ch2 , b3_out_ch3]
    b3_c_ksize4_a =[3,1,b3_out_ch3 , b3_out_ch4_a]
    b3_c_ksize4_b =[1,3,b3_out_ch3, b3_out_ch4_b]

    b3_w_conv1 , b3_b_conv1 = make_weights_biases('b3_'+layer_name+'_1','W','B',b3_c_ksize1 , device)
    b3_w_conv2 , b3_b_conv2 = make_weights_biases('b3_'+layer_name+'_2','W','B',b3_c_ksize2 , device)
    b3_w_conv3 , b3_b_conv3 = make_weights_biases('b3_'+layer_name+'_3','W','B',b3_c_ksize3 , device)
    b3_w_conv4_a , b3_b_conv4_a = make_weights_biases('b3_'+layer_name+'_4_a','W','B',b3_c_ksize4_a , device)
    b3_w_conv4_b , b3_b_conv4_b = make_weights_biases('b3_'+layer_name+'_4_b','W','B',b3_c_ksize4_b , device)

    b3_c_strides1=[1,1,1,1]
    b3_c_strides2=[1,1,1,1]
    b3_c_strides3=[1,1,1,1]
    b3_c_strides4_a=[1,1,1,1]
    b3_c_strides4_b=[1,1,1,1]

    b3_c_pooling1='SAME'
    b3_c_pooling2='SAME'
    b3_c_pooling3='SAME'
    b3_c_pooling4_a='SAME'
    b3_c_pooling4_b='SAME'
    b3_conv_info1=make_conv_info(x, b3_w_conv1, b3_b_conv1, b3_c_strides1, b3_c_pooling1)
    b3_layer1=convolution(b3_conv_info1 , place_info , bn_flag ,'b3_'+layer_name+'_L1' ,'xavier',restore_flag,  restore_path)
    
    b3_conv_info2=make_conv_info(b3_layer1, b3_w_conv2, b3_b_conv2, b3_c_strides2, b3_c_pooling2)
    b3_layer2=convolution(b3_conv_info2 , place_info , bn_flag ,'b3_'+layer_name+'_L2' ,'xavier',restore_flag,  restore_path)
    
    b3_conv_info3=make_conv_info(b3_layer2, w_conv3, b_conv3, c_strides3, c_pooling3)
    b3_layer3=convolution(b3_conv_info3 , place_info , bn_flag ,'b3_'+layer_name+'_L3' ,'xavier',restore_flag,  restore_path)
    
    b3_conv_info4=make_conv_info(b3_layer3, w_conv4, b_conv4, c_strides4, c_pooling4)
    b3_layer4=convolution(b3_conv_info4 , place_info , bn_flag ,'b3_'+layer_name+'_L4' ,'xavier',restore_flag,  restore_path)
    
    b3_conv_info5=make_conv_info(b3_layer4, w_conv5, b_conv5, c_strides5, c_pooling5)
    b3_layer5=convolution(b3_conv_info5 , place_info , bn_flag ,'b3_'+layer_name+'_L5' ,'xavier',restore_flag,  restore_path)


    #################################################################################  with tf.variable_scope(layer_name+'b3_layer5') as scope:    
    with tf.variable_scope(layer_name+'_end') as scope:    
        layer_concat = tf.concat(3 , [layer2_a,layer2_b,b1_layer2, b2_layer2,b3_layer5],name='concat')

    return layer_concat

In [29]:
def INCEPTION_REDUCTION_A(x , device,layer_name , place_info  , bn_flag  , restore_flag , restore_path):
    ####################################################################################
    """
    usage:
    x shape =[ n_batch , row , col , ch] 
    """
    print x.get_shape()
    in_ch=x.get_shape()[3]
    out_ch = 384
    c_ksize= [3,3,in_ch , out_ch]
    w_conv , b_conv =make_weights_biases(layer_name,'W','B',c_ksize,device)
    c_strides=[1,2,2,1]
    c_pooling='VALID'
    conv_info=make_conv_info(x, w_conv, b_conv, c_strides, c_pooling)
    layer=convolution(conv_info , place_info , bn_flag ,layer_name+'_L',restore_flag,  restore_path)

    ####################################################################################   
    b1_p_ksize=[1,3,3,1]
    b1_p_strides=[1,2,2,1]
    b1_p_pooling='VALID'
    with tf.variable_scope('b1_'+layer_name+'_L') as scope:
        b1_layer = tf.nn.max_pool(x,b1_p_ksize , b1_p_strides , b1_p_pooling, name='max_pool')
        b1_layer_relu = tf.nn.relu(b1_layer , name='relu')
    ####################################################################################
    b2_out_ch1 = 192; b2_out_ch2=288 ;b2_out_ch3 = 256;
    b2_c_ksize1=[1,1,in_ch,b2_out_ch1]
    b2_c_ksize2=[3,3,b2_out_ch1,b2_out_ch2]
    b2_c_ksize3=[3,3,b2_out_ch2,b2_out_ch3]
    
    b2_w_conv1 , b2_b_conv1 = make_weights_biases('b2_'+layer_name+'_1' ,'W' ,'B', b2_c_ksize1 , device)
    b2_w_conv2 , b2_b_conv2 = make_weights_biases('b2_'+layer_name+'_2' ,'W' ,'B',b2_c_ksize2 , device)
    b2_w_conv3 , b2_b_conv3 = make_weights_biases('b2_'+layer_name+'_3' ,'W' ,'B',b2_c_ksize3 , device)
    
    b2_c_strides1=[1,1,1,1]
    b2_c_strides2=[1,1,1,1]
    b2_c_strides3=[1,2,2,1]
    b2_c_pooling1='SAME'
    b2_c_pooling2='SAME'
    b2_c_pooling3='VALID'
    
    b2_conv_info1=make_conv_info(x, b2_w_conv1, b2_b_conv1, b2_c_strides1, b2_c_pooling1)
    b2_layer1=convolution(b2_conv_info1 , place_info , bn_flag ,'b2_'+layer_name+'_L1' ,restore_flag,  restore_path)
    
    b2_conv_info2=make_conv_info(b2_layer1, b2_w_conv2, b2_b_conv2, b2_c_strides2, b2_c_pooling2)
    b2_layer2=convolution(b2_conv_info2 , place_info , bn_flag ,'b2_'+layer_name+'_L2' ,restore_flag,  restore_path)
    
    b2_conv_info3=make_conv_info(b2_layer2, b2_w_conv3, b2_b_conv3, b2_c_strides3, b2_c_pooling3)
    b2_layer3=convolution(b2_conv_info3 , place_info , bn_flag ,'b2_'+layer_name+'_L3' ,restore_flag,  restore_path)
    
    ####################################################################################
    with tf.variable_scope(layer_name+'_end') as scope:    
        layer_concat=tf.concat(3 ,[layer ,b1_layer_relu ,b2_layer3] ,name='concat')

    
    
    return layer_concat
    

In [30]:
def INCEPTION_REDUCTION_B(x , device_,layer_name,n_nodes , place_info  , bn_flag  , restore_flag , restore_path):
    """
    
    """
    
    ####################################################################################
    in_ch=x.get_shape()[3]
    out_ch1=192 ; out_ch2=192;

    c_ksize1 =[1,1,in_ch , out_ch1]
    c_ksize2 =[3,3,out_ch1,out_ch2]
    w_conv1, b_conv1 = make_weights_biases(layer_name+'_1','W','B',c_ksize1,device)
    w_conv2, b_conv2 = make_weights_biases(layer_name+'_2','W','B',c_ksize2,device)
    
    c_strides1=[1,1,1,1]
    c_strides2=[1,2,2,1]
    
    c_pooling1='SAME'
    c_pooling2='VALID'
    conv_info1=make_conv_info(x, w_conv1, b_conv1, c_strides1, c_pooling1)
    layer1=convolution(conv_info1 , place_info , bn_flag ,layer_name+'_L1' ,'xavier',restore_flag,  restore_path)
    conv_info2=make_conv_info(layer1, w_conv2, b_conv2, c_strides2, c_pooling2)
    layer2=convolution(conv_info2 , place_info , bn_flag ,layer_name+'_L2' ,'xavier',restore_flag,  restore_path)
    ####################################################################################
    b1_p_ksize=[1,3,3,1]
    b1_p_strides=[1,2,2,1]
    b1_p_pooling='VALID'
    with tf.variable_scope('b1_'+layer_name+'L') as scope:
        b1_layer=tf.nn.max_pool(x ,b1_p_ksize , b1_p_strides , b1_p_pooling,name='max_pool')
        b1_layer_relu = tf.nn.relu(b1_layer , name='relu')
    ####################################################################################
    b2_out_ch1=256 ; b2_out_ch2 = 256 ; b2_out_ch3=320 ; b2_out_ch4=320;
    b2_c_ksize1 =[1,1,in_ch , b2_out_ch1]
    b2_c_ksize2 =[1,7,b2_out_ch1 , b2_out_ch2]
    b2_c_ksize3 =[7,1,b2_out_ch2 , b2_out_ch3]
    b2_c_ksize4 =[3,3,b2_out_ch3 , b2_out_ch4]
    
    b2_w_conv1, b2_b_conv1 = make_weights_biases('b2_'+layer_name+'_1','W' ,'B',b2_c_ksize1,device)
    b2_w_conv2, b2_b_conv2 = make_weights_biases('b2_'+layer_name+'_2','W' ,'B',b2_c_ksize2,device)
    b2_w_conv3, b2_b_conv3 = make_weights_biases('b2_'+layer_name+'_3','W' ,'B',b2_c_ksize3,device)
    b2_w_conv4, b2_b_conv4 = make_weights_biases('b2_'+layer_name+'_4','W' ,'B',b2_c_ksize4,device)
    
    b2_c_strides1=[1,1,1,1]
    b2_c_strides2=[1,1,1,1]
    b2_c_strides3=[1,1,1,1]
    b2_c_strides4=[1,2,2,1]
    
    b2_c_pooling1= 'SAME'
    b2_c_pooling2= 'SAME'
    b2_c_pooling3 = 'SAME'
    b2_c_pooling4 = 'VALID'
    
    b2_conv_info1=make_conv_info(x, b2_w_conv1, b2_b_conv1, b2_c_strides1, b2_c_pooling1)
    b2_layer1=convolution(b2_conv_info1 , place_info , bn_flag ,'b2_'+layer_name+'_L1' ,'xavier',restore_flag,  restore_path)
    
    b2_conv_info2=make_conv_info(b2_layer1, b2_w_conv2, b2_b_conv2, b1_c_strides2, b1_c_pooling2)
    b2_layer2=convolution(b2_conv_info2 , place_info , bn_flag ,'b2_'+layer_name+'_L2' ,'xavier',restore_flag,  restore_path)
    
    b2_conv_info3=make_conv_info(b2_layer2, b2_w_conv3, b2_b_conv3, b2_c_strides3, b2_c_pooling3)
    b2_layer3=convolution(b2_conv_info3 , place_info , bn_flag ,'b2_'+layer_name+'_L3' ,'xavier',restore_flag,  restore_path)

    b2_conv_info3=make_conv_info(b2_layer2, b2_w_conv4, b2_b_conv4, b2_c_strides4, b2_c_pooling4)
    b2_layer3=convolution(b2_conv_info4 , place_info , bn_flag ,'b2_'+layer_name+'_L4' ,'xavier',restore_flag,  restore_path)

    b2_conv_info3=make_conv_info(b2_layer2, b2_w_conv5, b2_b_conv5, b2_c_strides5, b2_c_pooling5)
    b2_layer3=convolution(b2_conv_info5 , place_info , bn_flag ,'b2_'+layer_name+'_L5' ,'xavier',restore_flag,  restore_path)

    with tf.variable_scope(layer_name+'_end') as scope:
        layer_concat_A=tf.concat(3,[layer2,b1_layer_relu,b2_layer3] ,name='concat')
    return layer_concat_A
    ####################################################################################

# FULLYCONNCECT

In [31]:
def FLAT(x , device_ , layer_name = "FLAT"):
    with tf.variable_scope(layer_name) as scope:
        with tf.device(device_):
            row=int(x.get_shape()[1])
            col=int(x.get_shape()[2])
            ch=int(x.get_shape()[3])

            res_x = tf.reshape(x , shape=[-1,row*col*ch] ,name='flat_layer');
            return res_x
            #connect fully connected layer 

In [32]:
def FC_A(x , dropout_rate ,n_classes , device_ , layer_name , restore_flag , restore_path):
    with tf.device(device_):    
        fully_ch1=1024; fully_ch2=1024;
        
        fc_ksize1=[x.get_shape()[1],fully_ch1]
        fc_ksize2=[fully_ch1,fully_ch2]

        w_fc1 ,b_fc1 = make_weights_biases(layer_name+'_1' , 'W' ,'B' ,fc_ksize1 ,  device_,'xavier',restore_flag , restore_path)
        w_fc2 ,b_fc2 = make_weights_biases(layer_name+'_2' , 'W' ,'B' ,fc_ksize2 ,  device_,'xavier',restore_flag , restore_path)
    
        h_fc1=tf.matmul(x, w_fc1 ,name='h_fc1')+b_fc1
        h_fc1=tf.nn.dropout(h_fc1 , dropout_rate , name='h_fc1_dropout')
        h_fc2=tf.matmul(h_fc1 , w_fc2 ,name='h_fc2')+b_fc2
        h_fc2=tf.nn.dropout(h_fc2 , dropout_rate,name='h_fc2_dropout')
        end_fc=h_fc2

        end_ksize=[end_fc.get_shape()[1] , n_classes]   
        w_end ,b_end = make_weights_biases('fc_end' , 'W' , 'B' ,end_ksize ,  device_)
        y_conv = tf.matmul(end_fc , w_end ,name=layer_name+'y_conv')+b_end

        print w_fc1.get_shape()
    return y_conv
    

In [33]:
def FC_B(x , n_classes , device_ , layer_name , restore_flag , restore_path):
    with tf.device(device_):
        end_ksize=[x.get_shape()[1] , n_classes]   
        w_end ,b_end = make_weights_biases\
        (layer_name , 'W','B' , end_ksize ,  device_ ,'xavier',restore_flag , restore_path)
        y_conv = tf.matmul(x , w_end , name=layer_name+'y_conv')+b_end
        return y_conv

In [34]:
def FC_C(x , dropout_rate,n_classes , device_ , layer_name , restore_flag , restore_path):
    with tf.device(device_):    
        fully_ch1=1024;
        
        fc_ksize1=[x.get_shape()[1],fully_ch1]

        w_fc1 ,b_fc1 = make_weights_biases(layer_name , 'W' ,'B' ,fc_ksize1 ,  device_,'xavier',restore_flag , restore_path)    
        h_fc1=tf.matmul(x, w_fc1 ,name='h_fc1')+b_fc1
        h_fc1=tf.nn.dropout(h_fc1 , dropout_rate , name='h_fc1_dropout')
        end_fc=h_fc1
        print end_fc
        end_ksize=[end_fc.get_shape()[1] , n_classes]   
        w_end ,b_end = make_weights_biases(layer_name+'_end' , 'W' , 'B' ,end_ksize ,  device_,'xavier',restore_flag , restore_path)
        y_conv = tf.matmul(end_fc , w_end , name=layer_name+'y_conv')+b_end

        print w_fc1.get_shape()
    return y_conv
    

In [35]:
def AVG_POOL(x , ksize , strides ,pooling,layer_name,device_):
    with tf.device(device_):
        layer=tf.nn.avg_pool(x,ksize ,strides,pooling,name = layer_name+"AVG_POOL")
    return layer

In [36]:
def MAX_POOL(x , ksize , strides,pooling,layer_name,device_):
    with tf.device(device_):
        layer=tf.nn.max_pool(x,ksize ,strides,pooling,name = layer_name+"MAX_POOL")
    print layer
    return layer
    

In [37]:
def DROPOUT(x ,keep_prob , layer_name , device_):
    with tf.device(device_):
        drop_x=tf.nn.dropout(x , keep_prob , name=layer_name+'dropout')
        return drop_x

# CONVOLUTION LAYER

In [38]:
def CONV_6(x , device_ , layer_name , n_nodes , kernel_size ,strides ,pooling):
    """ 
    k_row=kernel_size[0]  
    k_col=kernel_size[0] 
    
    """
    in_ch = x.get_shape()[3]
    k_row=kernel_size[0]  
    k_col=kernel_size[1] 
    
    
    out_ch1=n_nodes
    out_ch2=n_nodes
    out_ch3=n_nodes
    out_ch4=n_nodes
    out_ch5=n_nodes
    out_ch6=n_nodes
    
    c_ksize1=[k_row,k_col , in_ch   , out_ch1]
    c_ksize2=[k_row,k_col , out_ch1 , out_ch2]
    c_ksize3=[k_row,k_col , out_ch2 , out_ch3]
    c_ksize4=[k_row,k_col , out_ch3 , out_ch4]
    c_ksize5=[k_row,k_col , out_ch4 , out_ch5]
    c_ksize6=[k_row,k_col , out_ch5 , out_ch6]
    
    c_strides1=strides
    c_strides2=strides
    c_strides3=strides
    c_strides4=strides
    c_strides5=strides
    c_strides6=strides
    
    c_pooling1=pooling
    c_pooling2=pooling
    c_pooling3=pooling
    c_pooling4=pooling
    c_pooling5=pooling
    c_pooling6=pooling
    
    w_conv1 , b_conv1 =make_weights_biases(layer_name+str(1) , 'W1' ,'B1', c_ksize1 ,device_name = device_)
    w_conv2 , b_conv2= make_weights_biases(layer_name+str(2) , 'W2' ,'B2',c_ksize2 ,device_name = device_)
    w_conv3 , b_conv3= make_weights_biases(layer_name+str(3) , 'W3' ,'B3' ,c_ksize3 ,device_name = device_)
    w_conv4 , b_conv4= make_weights_biases(layer_name+str(4) , 'W4' ,'B4', c_ksize4 ,device_name = device_)
    w_conv5 , b_conv5= make_weights_biases(layer_name+str(5) , 'W5' ,'B5', c_ksize5 ,device_name = device_)
    w_conv6 , b_conv6= make_weights_biases(layer_name+str(6) , 'W6' ,'B6', c_ksize5 ,device_name = device_)

    with tf.variable_scope(layer_name+'conv1') as scope:
        layer1 = tf.nn.conv2d(x ,      w_conv1 , c_strides1 , c_pooling1 ,name='layer1')+b_conv1
        layer1 = tf.nn.relu(layer1,name='layer1_relu')
    with tf.variable_scope(layer_name+'conv2') as scope:
        layer2 = tf.nn.conv2d(layer1 , w_conv2 , c_strides2 , c_pooling2 , name='layer2')+b_conv2
        layer2 = tf.nn.relu(layer2,name='layer2_relu')
    with tf.variable_scope(layer_name+'conv2') as scope:
        layer3 = tf.nn.conv2d(layer2 , w_conv3 , c_strides3 , c_pooling3,name='layer3')+b_conv3
        layer3 = tf.nn.relu(layer3,name='layer3_relu')
    with tf.variable_scope(layer_name+'conv4') as scope:
        layer4 = tf.nn.conv2d(layer3 , w_conv4 , c_strides4 , c_pooling4,name='layer4')+b_conv4
        layer4 = tf.nn.relu(layer4,name='layer4_relu')
    with tf.variable_scope(layer_name+'conv5') as scope:
        layer5 = tf.nn.conv2d(layer4 , w_conv5 , c_strides5 , c_pooling5,name='layer5')+b_conv5
        layer5 = tf.nn.relu(layer5,name='layer5_relu')
    with tf.variable_scope(layer_name+'conv6') as scope:
        layer6 = tf.nn.conv2d(layer5 , w_conv6 , c_strides6 , c_pooling6,name='layer6')+b_conv6
        layer6 = tf.nn.relu(layer5,name='layer6_relu')

    return layer6

In [39]:
def CONV_5(x , device_ , layer_name , n_nodes , kernel_size ,strides ,pooling):
    """ 
    k_row=kernel_size[0]  
    k_col=kernel_size[0] 
    
    """
    in_ch = x.get_shape()[3]

    c_kernel1,c_kernel2,c_kernel3,c_kernel4,c_kernel5 = kernel_size  
    out_ch1,out_ch2,out_ch3,out_ch4,out_ch5=n_nodes

    c_ksize1=[c_kernel1[0],c_kernel1[1] , in_ch   , out_ch1]
    c_ksize2=[c_kernel2[0],c_kernel2[1] , out_ch1 , out_ch2]
    c_ksize3=[c_kernel3[0],c_kernel3[1] , out_ch2 , out_ch3]
    c_ksize4=[c_kernel4[0],c_kernel4[1] , out_ch3 , out_ch4]
    c_ksize5=[c_kernel5[0],c_kernel5[1] , out_ch4 , out_ch5]
    
    c_strides1,c_strides2,c_strides3,c_strides4,c_strides5=strides
    c_pooling1,c_pooling2,c_pooling3,c_pooling4,c_pooling5=pooling

    w_conv1 , b_conv1 =make_weights_biases(layer_name+str(1), 'W1' ,'B1' , c_ksize1 ,device_name = device_)
    w_conv2 , b_conv2= make_weights_biases(layer_name+str(2), 'W2' ,'B2' , c_ksize2 ,device_name = device_)
    w_conv3 , b_conv3= make_weights_biases(layer_name+str(3), 'W3' ,'B3' , c_ksize3 ,device_name = device_)
    w_conv4 , b_conv4= make_weights_biases(layer_name+str(4), 'W4' ,'B4' , c_ksize4 ,device_name = device_)
    w_conv5 , b_conv5= make_weights_biases(layer_name+str(5), 'W5' ,'B5' , c_ksize5 ,device_name = device_)

    with tf.variable_scope(layer_name+'conv1') as scope:
        layer1 = tf.nn.conv2d(x ,      w_conv1 , c_strides1 , c_pooling1 ,name='layer1')+b_conv1
        layer1 = tf.nn.relu(layer1,name='layer1_relu')
    with tf.variable_scope(layer_name+'conv2') as scope:
        layer2 = tf.nn.conv2d(layer1 , w_conv2 , c_strides2 , c_pooling2 , name='layer2')+b_conv2
        layer2 = tf.nn.relu(layer2,name='layer2_relu')
    with tf.variable_scope(layer_name+'conv2') as scope:
        layer3 = tf.nn.conv2d(layer2 , w_conv3 , c_strides3 , c_pooling3,name='layer3')+b_conv3
        layer3 = tf.nn.relu(layer3,name='layer3_relu')
    with tf.variable_scope(layer_name+'conv4') as scope:
        layer4 = tf.nn.conv2d(layer3 , w_conv4 , c_strides4 , c_pooling4,name='layer4')+b_conv4
        layer4 = tf.nn.relu(layer4,name='layer4_relu')
    with tf.variable_scope(layer_name+'conv5') as scope:
        layer5 = tf.nn.conv2d(layer4 , w_conv5 , c_strides5 , c_pooling5,name='layer5')+b_conv5
        layer5 = tf.nn.relu(layer5,name='layer4_relu')

    return layer5

In [40]:
def CONV_3(x , device_ , layer_name , n_nodes , kernel_size ,strides,pooling):
    """
    c_ksize1=[k_row,k_col , in_ch   , out_ch1]
    c_ksize2=[k_row,k_col , out_ch1 , out_ch2]
    c_ksize3=[k_row,k_col , out_ch2 , out_ch3]
    """
    
    in_ch = x.get_shape()[3]
    k_row=kernel_size[0]  
    k_col=kernel_size[1]     
    
    c_ksize1=[k_row,k_col , in_ch   , n_nodes]
    c_ksize2=[k_row,k_col , n_nodes , n_nodes]
    c_ksize3=[k_row,k_col , n_nodes , n_nodes]
    
    c_strides1=strides
    c_strides2=strides
    c_strides3=strides
        
    c_pooling1=pooling
    c_pooling2=pooling
    c_pooling3=pooling
    
    w_conv1 , b_conv1 =make_weights_biases(layer_name+str(1) , 'W1' ,'B1', c_ksize1 ,device_name = device_)
    w_conv2 , b_conv2= make_weights_biases(layer_name+str(2) , 'W2' ,'B2',c_ksize2 ,device_name = device_)
    w_conv3 , b_conv3= make_weights_biases(layer_name+str(3) , 'W3' ,'B3' ,c_ksize3 ,device_name = device_)
    
    with tf.variable_scope(layer_name+'conv1') as scope:
        layer1 = tf.nn.conv2d(x ,      w_conv1 , c_strides1 , c_pooling1 ,name='layer1')+b_conv1
        layer1 = tf.nn.relu(layer1,name='layer1_relu')
    with tf.variable_scope(layer_name+'conv2') as scope:
        layer2 = tf.nn.conv2d(layer1 , w_conv2 , c_strides2 , c_pooling2 , name='layer2')+b_conv2
        layer2 = tf.nn.relu(layer2,name='layer2_relu')
    with tf.variable_scope(layer_name+'conv2') as scope:
        layer3 = tf.nn.conv2d(layer2 , w_conv3 , c_strides3 , c_pooling3,name='layer3')+b_conv3
        layer3 = tf.nn.relu(layer3,name='layer3_relu')
    return layer3

In [41]:
def CONV_3_BN(x , device_ , layer_name , n_nodes , kernel_size ,strides,pooling,train_flag=True):
    """
    c_ksize1=[k_row,k_col , in_ch   , out_ch1]
    c_ksize2=[k_row,k_col , out_ch1 , out_ch2]
    c_ksize3=[k_row,k_col , out_ch2 , out_ch3]
    """
    
    in_ch = x.get_shape()[3]
    k_row=kernel_size[0]  
    k_col=kernel_size[1]     
    
    c_ksize1=[k_row,k_col , in_ch   , n_nodes]
    c_ksize2=[k_row,k_col , n_nodes , n_nodes]
    c_ksize3=[k_row,k_col , n_nodes , n_nodes]
    
    c_strides1=strides
    c_strides2=strides
    c_strides3=strides
        
    c_pooling1=pooling
    c_pooling2=pooling
    c_pooling3=pooling
    
    w_conv1 , b_conv1 =make_weights_biases(layer_name+str(1) , 'W1' ,'B1', c_ksize1 ,device_name = device_)
    w_conv2 , b_conv2= make_weights_biases(layer_name+str(2) , 'W2' ,'B2',c_ksize2 ,device_name = device_)
    w_conv3 , b_conv3= make_weights_biases(layer_name+str(3) , 'W3' ,'B3' ,c_ksize3 ,device_name = device_)
    
    
    beta1, gamma1 = tf.Variable()
    with tf.variable_scope(layer_name+'conv1') as scope:
        layer1 = tf.nn.conv2d(x ,      w_conv1 , c_strides1 , c_pooling1 ,name='layer1')+b_conv1
        layer1_BN = batch_norm(layer1,n_nodes ,train_flag,layer_name+'bn')
        layer1_relu = tf.nn.relu(layer1_BN,name='layer1_relu')
        
    with tf.variable_scope(layer_name+'conv2') as scope:
        layer2 = tf.nn.conv2d(layer1_relu , w_conv2 , c_strides2 , c_pooling2 , name='layer2')+b_conv2
        layer2_BN = batch_norm(layer2,n_nodes ,train_flag,layer_name+'bn2')
        layer2_relu = tf.nn.relu(layer2_BN,name='layer2_relu')
        
    with tf.variable_scope(layer_name+'conv2') as scope:
        layer3 = tf.nn.conv2d(layer2_relu , w_conv3 , c_strides3 , c_pooling3,name='layer3')+b_conv3
        layer3_BN=batch_norm(layer3,n_nodes ,train_flag,layer_name+'bn3')
        layer3_relu = tf.nn.relu(layer3_BN,name='layer3_relu')
    return layer3_relu

In [42]:
def CONV_1(x , device_ , layer_name , n_nodes , kernel_size ,strides, pooling,  place_info  , bn_flag  , restore_flag , restore_path):
    """
    c_ksize1=[k_row,k_col , in_ch   , out_ch1]
    c_ksize2=[k_row,k_col , out_ch1 , out_ch2]
    c_ksize3=[k_row,k_col , out_ch2 , out_ch3]
    """
    
    in_ch = x.get_shape()[3]
    out_ch=n_nodes
    c_ksize=[kernel_size[0], kernel_size[1] , in_ch, out_ch]
    c_pooling =pooling
    c_strides=strides # when downsampling featrue map , set strides parameter,[1,2,2,1]
    w_conv , b_conv =make_weights_biases(layer_name,'W' ,'B' , c_ksize ,device_ ,'xavier',restore_flag ,  restore_path )
    conv_info=make_conv_info(x, w_conv, b_conv, c_strides, c_pooling)
    layer1=convolution(conv_info , place_info , bn_flag , layer_name+'_L' , restore_flag , restore_path)   
    print layer_name , layer1.get_shape()
    return layer1

In [43]:
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [ atoi(c) for c in re.split('(\d+)', text) ]




In [44]:
def get_batch_list(folder_path):
    """
    return ret_train_img_list ,ret_train_lab_list 
    
    """
    list_files=os.walk(folder_path).next()[2]
    ret_train_img_list=[]
    ret_train_lab_list=[]
    np_train_imgs=[]
    np_train_labs=[]
    for i , ele in enumerate(list_files):

        if 'train'  in ele and 'img'in ele:
            ret_train_img_list.append(ele)
        elif 'train' in ele  and  'lab' in ele:
            ret_train_lab_list.append(ele)
    
    ret_train_img_list.sort(key=natural_keys)
    ret_train_lab_list.sort(key = natural_keys)
    print 'check match image and label '
    for i in range(len(ret_train_img_list)):
        print ret_train_img_list[i] , ret_train_lab_list[i]
    
    for i  in range(len(ret_train_img_list)):
        print str(i)+'th Image loading... waiting for minute....'
        np_train_imgs.append(np.load(folder_path+ret_train_img_list[i]))
        np_train_labs.append(np.load(folder_path+ret_train_lab_list[i]))
    
    return np_train_imgs ,np_train_labs 

In [45]:
def aug(np_img ,img_row ,img_col , color_ch, crop_img_row , crop_img_col , label):
    
    n_img = np.shape(np_img)[0] 
    n_ret_img = n_img*(img_row - crop_img_row) * (img_col - crop_img_col)*2
    ret_images = np.zeros([n_ret_img ,crop_img_row , crop_img_col,color_ch])
    len_label= np.shape(label)[1]
    ret_labels =  np.zeros([n_ret_img  ,len_label])
    #print "n_augmented image size : " , n_ret_img 
    #print "n classes :", len_label
    #copy label in factor by 2014
    
    if len(np.shape(np_img))==2:
        np_img=np.reshape(np_img , newshape = [np.shape(np_img)[0] , img_row , img_col ,color_ch])
        print np.shape(np_img)
    for n  in range(len(np_img)):
        ret_labels[n*2 , : ] = label[n,:]
        ret_labels[n*2+1 , : ] = label[n,:]
        for r in range(img_row - crop_img_row):
            for c in range(img_col - crop_img_col):
                
                cropped_img = np_img[n, r:crop_img_row +r , c:crop_img_col+c ,: ]
                
                ret_images[n*2,:,:,:]=cropped_img  
                ret_images[(n*2+1) , :,:,:] =np.fliplr(cropped_img )

    
    return ret_images ,ret_labels



In [46]:
def extract_test_img(np_img ,img_row ,img_col , color_ch, crop_img_row , crop_img_col ):
    left_top =(0,0)
    right_top =(  img_row  - crop_img_row  , 0 )
    center =  ((img_row  - crop_img_row)/2  , (img_col - crop_img_row)/2)
    left_buttom = (0,(img_col - crop_img_row)/2 )
    right_buttom =  (img_row  - crop_img_row , img_col - crop_img_row)
    
    left_top_images  = np_img[: , left_top[0]:crop_img_row+left_top[0] , left_top[1] : crop_img_col+left_top[1] , :  ]
    right_top_images = np_img[: , right_top[0]:crop_img_row +right_top[0], right_top[1] : crop_img_col +right_top[1], :  ]
    center_images    = np_img[: , center[0]:crop_img_row +center[0], center[1] : crop_img_col +center[1], :  ]
    left_buttom_images=np_img[: , left_buttom[0]:crop_img_row +left_buttom[0], left_buttom[1] : crop_img_col +left_buttom[1], :  ]
    right_buttom_images= np_img[: , right_buttom[0]:crop_img_row+right_buttom[0] , right_buttom[1] : crop_img_col +right_buttom[1] , :  ]

    
        
    return left_top_images , right_top_images , center_images , left_buttom_images , right_buttom_images 



In [47]:
def TRAIN_STRUCTURE_A(y_conv , y_ , device_,learning_rate ):
    """
    Return Value : cost , train_step ,correct_prediction , accuracy 
    
    """
    with tf.device(device_):
    #sm_conv= tf.nn.softmax(y_conv)
        #cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))

        softmax=tf.nn.softmax(y_conv)
        pred_cls = tf.argmax(softmax , axis = 1)
        #regular=0.01*(tf.reduce_sum(tf.square(y_conv)))
        cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits( y_conv, y_))
    with tf.device(device_):
        #cost = cost+regular
        train_step = tf.train.AdamOptimizer(learning_rate).minimize(cost) #1e-4
        with tf.name_scope("accuracy"):
            with tf.name_scope('correct_prediction'):
                correct_prediction = tf.equal(tf.argmax(y_conv,1) ,tf.argmax(y_,1))
            with tf.name_scope('accuracy'):
                accuracy = tf.reduce_mean(tf.cast(correct_prediction , "float")) 
    tensor_info ={}
    tensor_info['cost']=cost
    tensor_info['train_step']=train_step
    tensor_info['correct_prediction']=correct_prediction
    tensor_info['accuracy']=accuracy
    tensor_info['softmax']=softmax
    tensor_info['pred_cls'] = pred_cls
    return tensor_info

In [48]:
def START_SESS():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth=True
    sess = tf.Session(config=config)
    sess.run(tf.initialize_all_variables())
    return sess 


# File Terminal Open

In [49]:
#dirname = '/home/ncc/notebook/mammo/result/'
def make_logdir(dirname):
  

    count=0
    while(True):
        if not os.path.isdir(dirname):
            os.mkdir(dirname)
            break
        elif not os.path.isdir(dirname + str(count)):
            dirname=dirname+str(count)
            os.mkdir(dirname)
            break
        else:
            count+=1
    print 'it is recorded at :'+str(count)

    f=open(dirname+"/log.txt",'w')
    return f,dirname

In [50]:

def write_acc_loss(step , val_acc , val_loss ,file_path):
    """
    fp = File Pointer
    """
    str_ = 'step:\t'+str(step)+'\tacc:\t'+str(acc) +'\tloss:\t'+str(loss)+'\n'
    fp.write(str_)
    

In [51]:
def load_TVT(divide_flag,file_locate):
    if divide_flag == False:
        train_img=np.load(file_locate+'train_img.npy');
        train_lab=np.load(file_locate+'train_lab.npy');
        val_img= np.load(file_locate+'val_img.npy');
        val_lab = np.load(file_locate+'val_lab.npy');
        test_img=np.load(file_locate+'test_img.npy');
        test_lab=np.load(file_locate+'test_lab.npy');

        print "Training Data Image",np.shape(train_img)
        print "Training Data Label",np.shape(train_lab)
        print "Test Data Label",np.shape(test_lab)
        print "Test Data Image",np.shape(test_img)
        print "val Data Label" , np.shape(val_lab)
        print "val Data Image" , np.shape(val_img)
        
        n_train= np.shape(train_img)[0]
        n_train_lab = np.shape(train_lab)[0]
        return train_img ,train_lab,val_img,val_lab,test_img,test_lab
    if divide_flag == True:
        print '트레이닝 파일이 여러개로 분할되어 있습니다. 분할된 트레이닝 파일에 대한 조치가 필요합니다'
        train_images, train_labels =get_batch_list(file_locate)
        #print train_images ,train_labels
        val_img= np.load(file_locate+'val_img.npy');
        val_lab = np.load(file_locate+'val_lab.npy');
        test_img=np.load(file_locate+'test_img.npy');
        test_lab=np.load(file_locate+'test_lab.npy');
        print "the number of training image batch",len(train_images)
        print "the number of training label batch",len(train_labels)
        print "training Data Label",np.shape(train_labels[0])
        print "training Data Image",np.shape(train_images[0])
        print "Test Data Label",np.shape(test_lab)
        print "Test Data Image",np.shape(test_img)
        print "val Data Label" , np.shape(val_lab)
        print "val Data Image" , np.shape(val_img)
        return train_images, train_labels,val_img,val_lab,test_img,test_lab

In [52]:
def save_weights_biases(sess, save_folder):
    
    trainable_list=tf.trainable_variables()
    save_paths=[]
    for i,ele in enumerate(trainable_list): 
        #print ele
        #print ele.name.replace('/' , '_')
        np_=sess.run(ele)
        #print 'name:',ele.name
        np.save(save_folder+ele.name.replace('/' , '_').replace(':0' , '') , np_ )
 
    print "model was saved"
        

In [53]:
def validate(sess,img , lab ,tensor_info , place_info,tensorboard_info=None , step=None): #default
    """
    return val_acc,  val_loss, train_acc, train_loss
    tensorboard_info['writer']
    tensorboard_info['merge']    
    """    
    #print 'validate tensorboard' , tensorboard_info
    if tensorboard_info ==None: #텐서 보드를 비활성 합니다.
        print 'No tensorboard info'
        acc ,loss = sess.run( [tensor_info['accuracy'],tensor_info['cost']] ,\
                             feed_dict={place_info['x_']:img , place_info['y_']: lab , place_info['keep_prob']: 1.0})    
    else: # 텐서보드를 활성화 합니다
        print 'tensorboard info'
        summary,acc,loss = sess.run([tensorboard_info['merge'],tensor_info['accuracy'],tensor_info['cost']] ,\
                                    feed_dict={place_info['x_']:img , place_info['y_']:lab , place_info['keep_prob']: 1.0})        
        tensorboard_info['writer'].add_summary(summary , step)
        
        
    return acc,loss

In [54]:
  
def validate_extract_imgs(val_img , val_lab , train_img , train_lab ):
    """
    extract patch from ori-image
    """
    color_ch = in_ch
    val_images  =extract_test_img(val_img , 128 , 128 , color_ch   ,crop_img_row =118 , crop_img_col =118 )
    train_images=extract_test_img(train_img ,128 , 128 , color_ch   ,crop_img_row =118 , crop_img_col =118 )
    val_acc, val_loss =validate_from_images(sess, val_images , val_lab , accuracy ,cost )
    train_acc , train_loss =validate_from_images(sess, train_images , train_lab ,accuracy , cost)
    return val_acc , val_loss, train_acc ,train_loss
    


In [55]:
def aug_8_times(x):
    """
    
    
    x shape is [n_batch , row ,col , color_ch ]
    x type is numpy 
    this code need too many time to run 
    we should find solution using parallel method to less run time maybe 
    
    return x,np_rot90,np_rot180,np_rot270,lr_x,np_lr_rot90 ,np_lr_rot180 , np_lr_rot270 
    
    """

    n_batch,row,col,ch=np.shape(x)
    lr_x = np.flipud(x)

    np_rot90 =np.zeros(shape =[n_batch , row ,col ,ch] )
    np_rot180=np.zeros(shape =[n_batch , row ,col ,ch] )
    np_rot270=np.zeros(shape =[n_batch , row ,col ,ch] )

    np_lr_rot90 =np.zeros(shape =[n_batch , row ,col ,ch] )
    np_lr_rot180=np.zeros(shape =[n_batch , row ,col ,ch] )
    np_lr_rot270=np.zeros(shape =[n_batch , row ,col ,ch] )
    
    

    for batch_ind in range(n_batch):

        rot90=np.rot90(x[batch_ind,:,:,:])
        rot180=np.rot90(rot90)
        rot270=np.rot90(rot180)

        np_rot90[batch_ind,:,:,:] = rot90
        np_rot180[batch_ind,:,:,:]=rot180
        np_rot270[batch_ind,:,:,:]=rot270

        lr_rot90=np.rot90(lr_x[batch_ind,:,:,:])
        lr_rot180=np.rot90(lr_rot90)
        lr_rot270=np.rot90(lr_rot180)

        np_lr_rot90[batch_ind,:,:,:]=lr_rot90
        np_lr_rot180[batch_ind,:,:,:]=lr_rot180
        np_lr_rot90[batch_ind,:,:,:]=lr_rot270
    return x,np_rot90,np_rot180,np_rot270,lr_x,np_lr_rot90 ,np_lr_rot180 , np_lr_rot270 
    

In [56]:
def OPEN_TENSORBOARD(sess,tensor_info , logdir ):
    
    tensorboard_info={}
    tb_acc=tf.summary.scalar("accuarcy" ,tensor_info['accuracy'] )
    tb_loss=tf.summary.scalar("loss" ,tensor_info['cost'] )
    tb_merge = tf.summary.merge_all()
    writer=tf.train.SummaryWriter(logdir , sess.graph )
    sess.run(tf.global_variables_initializer())
    
    
    tensorboard_info['writer']=writer
    tensorboard_info['merge'] =tb_merge
  
    return tensorboard_info

In [57]:
def aug_crop(np_img ,img_row ,img_col , color_ch, crop_img_row , crop_img_col , label):
    n_img = np.shape(np_img)[0] 
    n_ret_img = n_img*(img_row - crop_img_row) * (img_col - crop_img_col)*2
    ret_images = np.zeros([n_ret_img ,crop_img_row , crop_img_col,color_ch])
    len_label= np.shape(label)[1]
    ret_labels =  np.zeros([n_ret_img  ,len_label])
    #print "n_augmented image size : " , n_ret_img 
    #print "n classes :", len_label
    #copy label in factor by 2014

    if len(np.shape(np_img))==2:
        np_img=np.reshape(np_img , newshape = [np.shape(np_img)[0] , img_row , img_col ,color_ch])
        print np.shape(np_img)
    for n  in range(len(np_img)):
        ret_labels[n*2 , : ] = label[n,:]
        ret_labels[n*2+1 , : ] = label[n,:]
        for r in range(img_row - crop_img_row):
            for c in range(img_col - crop_img_col):

                cropped_img = np_img[n, r:crop_img_row +r , c:crop_img_col+c ,: ]

                ret_images[n*2,:,:,:]=cropped_img  
                ret_images[(n*2+1) , :,:,:] =np.fliplr(cropped_img )


    return ret_images ,ret_labels
 


In [58]:
def save_result(dirname , result):
    """
     list_val_acc , list_val_loss , list_train_acc , list_train_loss
    """
    np_val_acc=np.asarray(result[0])
    np.save(dirname+'/val_acc' , np_val_acc)
    np_val_loss=np.asarray(result[1])
    np.save(dirname+'/val_loss' , np_val_loss)
    
    np_train_acc=np.asarray(result[2])
    np.save(dirname+'/train_acc' , np_train_acc)
    np_train_loss=np.asarray(result[3])
    np.save(dirname+'/train_loss' , np_train_loss)    

In [59]:
def load_val(val_imgs = None,  val_labs=None):
    """
    사용되는 글로벌 함수:
    1.flie_locate
    2.place_infro (divide_images)
    """
    if val_imgs == None and val_labs ==None: 
        start_time=time.time()
        val_img=np.load(file_locate + 'val_img.npy')
        val_lab=np.load(file_locate + 'val_lab.npy')
        val_imgs, val_labs = divide_images(val_img ,val_lab)
        end_time=time.time()
        print "validate images , labels was loaded and divided with batch_size"
        print "batch_size : ",batch_size
        return val_imgs, val_labs
    elif val_imgs != None and val_labs !=None:
        return val_imgs  , val_labs

    
def load_test(test_imgs = None , test_labs=None):
    
    """    
    사용되는 글로벌 함수:
    1.flie_locate
    2.place_infro (divide_images)
    """
    if test_imgs == None and test_labs ==None: 

        start_time=time.time()
        test_img=np.load(file_locate+'test_img.npy')
        test_lab=np.load(file_locate+'test_lab.npy')
        val_imgs, val_labs = divide_images(val_img ,val_lab)
        end_time=time.time()
        print "test images , labels was loaded and divided with batch_size"
        print "batch_size : ",batch_size
        print "loading time" , end_time-start_time
        return test_imgs, test_labs
    elif test_imgs != None and test_labs !=None:
        return test_imgs  , test_labs

def load_train(b_ind=None):
    if b_ind==None:
        b_ind=random.randrange(0,n_train)
    train_img=np.load(file_locate+'train_'+str(b_ind)+'_img.npy')
    train_lab=np.load(file_locate+'train_'+str(b_ind)+'_lab.npy')
    return train_img , train_lab ,b_ind

def load_train_all(start=None , end=None , arg_list=None):
    pool=Pool()
    train_imgs=[]
    train_labs=[]
    if arg_list!=None:
        list_ = arg_list
    elif start==None and end ==None:
        list_=range(n_train)    
    elif start!=None and end !=None:
        list_=range(start , end)
    
    for train_img , train_lab ,b_ind in pool.imap(load_train ,list_):
        train_imgs.append(train_img)
        train_labs.append(train_lab)
        msg='progress - {0}/{1}'.format(str(b_ind),str(n_train))
        sys.stdout.write(msg)
        sys.stdout.flush()
        
    return train_imgs , train_labs

def make_acc_step_log(folder_path):
    f=open(folder_path+'step_max_acc.txt' , 'w')
    msg='step\tacc\n'
    f.write(msg,)
    msg='{0}\t{1}\n'.format(str(0),str(0))
    f.write(msg)
    f.close()
def make_folder(folder_path='./'):
    # make folder that take accuracy , loss 
    # if folder is already existed , load folder. 
    up_to_date_folder=folder_path+'up_to_date/'
    interrupt_folder=folder_path+'Interrupt/'
    best_acc_folder=folder_path+'best_acc/'
    if os.path.isdir(up_to_date_folder) != True:
        os.mkdir(up_to_date_folder)
        make_acc_step_log(up_to_date_folder)
    if os.path.isdir(interrupt_folder)!=True:
        os.mkdir(interrupt_folder)
        make_acc_step_log(interrupt_folder)
    if os.path.isdir(best_acc_folder)!=True:
        os.mkdir(best_acc_folder)
        make_acc_step_log(best_acc_folder)

def load_step_acc(folder_path):
    try:
        f=open(folder_path+'step_max_acc.txt' , 'r')
        step ,max_acc=f.readlines()[-1].split('\t')
        
        print "global_step and max_acc was restore!"
        print "global step:",step
        print "Max acc:" ,max_acc
        return step , max_acc

    except IOError as ioe:
        print 'global_step or maxacc cant found in'+folder_path
        print 'Initialize global_step and max_acc to zero'
        step=0 ;acc=0
        return step , acc  
    except Exception as ex:
        print 'global_step or maxacc cant found in'+folder_path
        print 'Initialize global_step and max_acc to zero'
        step=0 ;acc=0
        return step , acc  
        
#def write_acc_loss()

def divide_images(images , labels):
    """
    return list_images,list_labels
    """
    n_divide=len(images)/batch_size

    list_images=[]
    list_labels=[]
    for ind in range(n_divide):
        # j*batch_size :(j+1)*batch_size
        image =images[ ind*batch_size :(ind+1)*batch_size] 
        label =labels[ ind*batch_size :(ind+1)*batch_size]
        list_images.append(image)
        list_labels.append(label)

    #right above code have to modify
    image = images[ -batch_size :  ] 
    label = labels[ -batch_size :  ]
    list_images.append(image)
    list_labels.append(label)
    return list_images,list_labels

def validate_from_images(images , labels ,tensor_info, place_info ,correct_pred=False,tensorboard_info = None ):    
    """
    input : x-

    x type  : numpy 
    x shape : [n , row , col , ch]
    return  acc,  loss


    2017/2/16
    왜 tensorboard 에 기록이 안되나 했더니 여기서 문제가 있었다.3
    validate_from_images에서는 기록이 안된다.
    기록할수 있는 방법을 찾아야 겠다
    내 생각에는 기존의 방식과 다른 tensorboard  객체를 만들고 거기다가 기록해야 할 것 같다.
    
    사용되는 글로벌 함수
    place_info 
    """    
    acc_list=[]
    loss_list=[]
    images_labels=zip(images,labels)
    temp_correct_list = []
    for img_ind ,(img,lab)  in enumerate(images_labels):
        acc ,loss , correct_prediction= sess.run( [tensor_info['accuracy'],tensor_info['cost'] , tensor_info['correct_prediction']],\
                             feed_dict={place_info['x_']:img , place_info['y_']: lab , place_info['keep_prob']: 1.0  , place_info['phase_train']:False})
        temp_correct_list.extend(correct_prediction)
        acc_list.append(acc)
        loss_list.append(loss)
    correct_prediction=temp_correct_list
    acc_list=np.asarray(acc_list)
    loss_list=np.asarray(loss_list)
    acc=np.mean(acc_list)
    loss=np.mean(loss_list)
    #print 'validate tensorboard' , tensorboard_info

    if correct_pred == True:
        #print 'correct_pred is True'
        return acc , loss , correct_prediction
    elif correct_pred == False :
        return  acc,  loss

def TRAINING(step,batch_xs , batch_ys , tensor_info ,place_info):
    msg = '\r-Progress : {0}/{1}'.format( str(step%check_point), str(check_point))
    sys.stdout.write(msg)
    sys.stdout.flush()
    #print '---------------------------- --------training start{0}------------------------------------'.format(step)    
    sess.run(tensor_info['train_step'] ,feed_dict={place_info['x_']: batch_xs, \
                                                   place_info['y_']:batch_ys , place_info['keep_prob'] : 0.5 , place_info['phase_train'] :True})
def VALIDATE(val_imgs,val_labs,tensor_info,place_info):
    start_time=time.time()
    val_imgs,val_labs=divide_images(val_imgs , val_labs)
    print np.shape(val_imgs)
    val_acc, val_loss =validate_from_images(val_imgs ,val_labs ,tensor_info ,place_info) # 이걸 여러개의 쓰레드로 나눠서 실행해야한다
    print time.time()-start_time
    return val_acc , val_loss

def TESTING(test_imgs, test_labs , tensor_info , place_info , training=True):
    test_imgs,test_labs=divide_images(test_imgs, test_labs)
    test_acc, test_loss =validate_from_images(test_imgs ,test_labs ,tensor_info ,place_info)
    print "test accuracy {0} , test loss {1}".format(test_acc, test_loss)
def save_weights_biases(step , max_acc , val_acc, save_folder ):
    if  val_acc  > max_acc:
        f=open(save_folder+'step_max_acc.txt','a')
        msg='{0}\t{1}\n'.format(step , max_acc)
        f.write(msg)
        max_acc = val_acc  
        trainable_list=tf.trainable_variables()
        save_paths=[]
        for i,ele in enumerate(trainable_list): 
            #print ele
            #print ele.name.replace('/' , '_')
            np_=sess.run(ele)
            #print 'name:',ele.name
            np.save(save_folder+ele.name.replace('/' , '_').replace(':0' , '') , np_ )

        print "model was saved"
        return max_acc
    else:
        return max_acc
"""
    def show_histo(save_folder,global_step):
    if show_histo_flag:
    if os.path.isdir(save_folder+'histogram/') == False:
    os.mkdir(save_folder+'histogram/')
    show_histo(save_folder+'histogram/' , step)
    print "making histogram"
    n_trainable=len(tf.trainable_variables())
    plot_row=int(math.ceil((math.sqrt(n_trainable))))

    plot_col=plot_row 
    print plot_col , plot_row
    fig , axes  = plt.subplots(plot_row ,plot_col ,figsize=(60, 60))

    for i , tensor in enumerate(tf.trainable_variables()):
        if 'W' in tensor.name or 'B' in tensor.name:
            values=sess.run(tensor.name)
            values_np = np.asarray(values).flatten()
            axes.flat[i].hist(values_np)
            axes.flat[i].set_xlabel(tensor.name)
    plt.savefig(save_folder+'/weights_and_biases_'+str(global_step)+'.png')
    print "saved histogram! "
    plt.close()
"""
def interrupt_mgs():
    print "keyboard Interupted!"
    print("--- Training Time : %s ---" % (time.time() - start_time))
    train_time="--- Training Time : ---:\t" +str(time.time() - start_time)
    print "now start validate Test set by model Don't stop! waiting for a minute..."

    step_np[0]=step
    maxacc_np[0] = max_acc
    print "now we start saving accuracy and loss"
    print "training acc,loss Validation acc and loss was saved!"        

def show_acc_loss( step , val_acc , val_loss):
    print("step %d , validation  accuracy %g" %(step,val_acc))
    print("step %d , validation loss : %g" %(step,val_loss))

def next_batch(batch_size,*args):
    """
    list_number : 
    Type : list 
    N.B. the argument list_restricNum is extracted in turn  
    from each kwargs element
    list_clsNum: 
    Type : list 
    N.B. the argument list_clsNum point at number which 
    each args list is taken   
    """
    #print batch_size
    batch_count=0
    imgs_list=[]
    for i,thing in enumerate(args):
        imgs_list.append(thing)
    #print 'the number of classes : ',n_classes
    ret_imgs=[]
    ret_cls=[]
    for i in range(batch_size):    
        cls_idx=random.randint(0,n_classes-1)
        imgs=imgs_list[cls_idx]
        imgs=np.asarray(imgs)
        n_imgs =np.shape(imgs)[0]
        ind=random.randint(0,n_imgs-1)
        img=imgs[ind]
        #print np.shape(img)
        ret_imgs.append(img)
        ret_cls.append(cls_idx)
    ret_imgs=np.asarray(ret_imgs)
    ret_cls=np.asarray(ret_cls)
    ret_labs=cls2onehot(n_classes,ret_cls)
    return ret_imgs , ret_labs

def next_random_batch(image , label ):
    indices=random.sample(range(np.shape(image)[0])  , batch_size)
    batch_x = image[indices]
    batch_y= label[indices]
    return batch_x, batch_y

def next_batch_list(image_list , label_list , selected_list= None ):
    """
    image_list shape taken 5 shape : [n_ , batch , row , col , in_ch]
    3rd param 'list_' taken list that include which number run
    """
    if selected_list != None:
        random.shuffle(selected_list)
        ind=selected_list[0]
        #print 'selected_list'
        #print 'ind',ind
    elif selected_list == None:
        ind=np.random.randint(len(image_list))
    batch_x , batch_y=next_batch(image_list[ind] , label_list[ind] )
    return batch_x, batch_y


In [60]:
def VALIDATE_FROM_TRAIN(indices):
    acc_list=[]
    loss_list=[]
    correct_list=[]
    n_batch , row , col , input_ch =np.shape(train_imgs_list[0])
    for i in indices:
        imgs, labs =divide_images(train_imgs_list[i] ,train_labs_list[i] )
        acc ,loss , correct_=validate_from_images(imgs , labs, tensor_info , place_info , correct_pred=True)
        acc_list.append(acc)
        loss_list.append(loss)
        correct_list.extend(correct_)
        
    share =  len(train_imgs_list[i])/ batch_size 
    remainder = len(train_imgs_list[i])%batch_size
    temp_correct_=[]
    for k in range(len(indices)):
        list_=correct_list[ (k*n_batch) + k*(n_batch - remainder)  : (k+1)*n_batch + k*(n_batch - remainder) ]
        temp_correct_.extend(list_)
    correct_list=np.asarray(temp_correct_)
    acc=np.mean(correct_list)
    print len(correct_list)
    return acc , loss 

In [ ]:
def summary(log_path,save_path):
    

In [61]:
check_point=100
step=0
def BATCH_TRAINING_RANDOM(maxiter, tensor_info, place_info,save_folder,restore_path,tensorboard_info=None , val_img_lab =None , test_img_lab=None):
    try:
        max_acc=0
        make_folder(save_folder)
        start_step,max_acc=load_step_acc(restore_path)
        #val_imgs,val_labs=load_val()
        for i in range(int(start_step), maxiter):
            step=i
            if step%check_point==0: #여기 if loop에서 validate을 합니다     
                cat_val_acc,cat_val_loss=VALIDATE(ret_imgs[-300:], ret_labs[-300:],tensor_info , place_info)
                #ret_acc,ret_loss=VALIDATE(ret_imgs[-100:], ret_labs[-100:],tensor_info , place_info)
                #gla_acc,gla_loss=VALIDATE(gla_imgs[-100:], gla_labs[-100:],tensor_info , place_info)
                normal_1_acc,normal_1_loss=VALIDATE(normal_1_imgs[-300:], normal_1_labs[-300:],tensor_info , place_info)
                total_acc=(cat_val_acc +normal_1_acc)/2
                total_loss=(cat_val_loss +normal_1_loss)/2
                
                show_acc_loss(step,cat_val_acc , cat_val_loss) 
                #show_acc_loss(step,ret_acc,ret_loss) 
                #show_acc_loss(step,gla_acc,gla_loss) 
                show_acc_loss(step,normal_1_acc,normal_1_loss) 

                #val_acc=float(val_acc)
                #max_acc=float(max_acc)
                #if val_acc > max_acc:
                #    print 'val_acc > max_acc'
                #    max_acc = val_acc
                #print 'max_acc', max_acc
            batch_xs ,batch_ys=next_batch(batch_size , ret_imgs[:-300],normal_1_imgs[:-300])
            TRAINING(step,batch_xs , batch_ys,tensor_info,place_info)
        test_imgs, test_labs=load_test()
        test_acc, test_loss=TESTING(test_imgs, test_labs,tensor_info)
    except KeyboardInterrupt as kbie:
        interrupt_mgs()

In [62]:
save_folder='./LAB_1/'
restore_path='./LAB_1/best_acc/'
restore_flag=False
device_='/gpu:1'
bn_flag=True

In [63]:
images=tf.map_fn (lambda image: pre_processing(image) ,x_)
print images.get_shape()
PRE=PRE_LAYER(x_ , device_,'PRE',64 , place_info  , bn_flag  , restore_flag , restore_path)
A=STEM_A(PRE , device_,'STEM_A',(32,32,64,96) , place_info  , bn_flag  , restore_flag , restore_path)
B=STEM_B(A , device_,'STEM_B', place_info  , bn_flag  , restore_flag , restore_path)
C=STEM_C(B , device_,'STEM_C' , place_info  , bn_flag  , restore_flag , restore_path)
D=INCEPTION_MODULE_A(C, device_,'MODUEL_A_a',place_info ,bn_flag, restore_flag , restore_path)
#E=INCEPTION_MODULE_A(D, device_,'MODUEL_A_b',place_info ,bn_flag, restore_flag , restore_path)
#F=INCEPTION_MODULE_A(E, device_,'MODUEL_A_c',place_info ,bn_flag, restore_flag , restore_path)
#G=INCEPTION_MODULE_A(F, device_,'MODUEL_A_d',place_info ,bn_flag, restore_flag , restore_path)
#H=INCEPTION_REDUCTION_A(G,device_,'REDUCT_A_a',place_info ,bn_flag, restore_flag , restore_path)
#H=INCEPTION_MODULE_A(G, device_,'MODUEL_A_e',place_info ,bn_flag, restore_flag , restore_path)
flat_ = FLAT(D , device_ , layer_name='FLAT_')
y_conv=FC_C(flat_ , 0.5 , n_classes , device_ , 'FC_C' , restore_flag , restore_path)
print y_conv
tensor_info=TRAIN_STRUCTURE_A(y_conv , y_ , device_ , learning_rate = 0.001)

(30, 288, 288, 3)
PRE make weigths and biases
(30, 75, 75, 64)
STEM_A_1 make weigths and biases
STEM_A_2 make weigths and biases
STEM_A_3 make weigths and biases
STEM_A_4 make weigths and biases
STEM_B_1 make weigths and biases
STEM_B_2 make weigths and biases
STEM_B_3 make weigths and biases
STEM_B_4 make weigths and biases
b_STEM_B_1 make weigths and biases
b_STEM_B_2 make weigths and biases
STEM_C_1 make weigths and biases
MODUEL_A_a_1 make weigths and biases
MODUEL_A_a_2 make weigths and biases
b1_MODUEL_A_a make weigths and biases
b2_MODUEL_A_a make weigths and biases
b3_MODUEL_A_a_1 make weigths and biases
b3_MODUEL_A_a_2 make weigths and biases
b3_MODUEL_A_a_3 make weigths and biases
Tensor("MODUEL_A_a_end/concat:0", shape=(30, 15, 15, 384), dtype=float32)
FC_C make weigths and biases
Tensor("h_fc1_dropout/mul:0", shape=(30, 1024), dtype=float32, device=/device:GPU:1)
FC_C_end make weigths and biases
(86400, 1024)
Tensor("add_1:0", shape=(30, 2), dtype=float32, device=/device:GP

In [64]:
sess=START_SESS()

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [65]:
#tensorboard_info=OPEN_TENSORBOARD(sess ,tensor_info ,logdir) for tensorboard slow down ?
#fp,dirname=make_logdir(dirname)
BATCH_TRAINING_RANDOM(100000 ,tensor_info , place_info ,save_folder , restore_path )

global_step and max_acc was restore!
global step: 0
Max acc: 0

(11, 30, 300, 300, 3)
2.57384181023
(11, 30, 300, 300, 3)
0.718773126602
step 0 , validation  accuracy 0.651515
step 0 , validation loss : 0.771799
step 0 , validation  accuracy 0.333333
step 0 , validation loss : 1.67859
-Progress : 99/100(11, 30, 300, 300, 3)
0.737827062607
(11, 30, 300, 300, 3)
0.705290079117
step 100 , validation  accuracy 0.948485
step 100 , validation loss : 4.70848
step 100 , validation  accuracy 0.0272727
step 100 , validation loss : 135.468
-Progress : 99/100(11, 30, 300, 300, 3)
0.694058895111
(11, 30, 300, 300, 3)
0.694045066833
step 200 , validation  accuracy 0.687879
step 200 , validation loss : 4.1239
step 200 , validation  accuracy 0.284849
step 200 , validation loss : 14.5703
-Progress : 99/100(11, 30, 300, 300, 3)
0.741234064102
(11, 30, 300, 300, 3)
0.707885980606
step 300 , validation  accuracy 0.0878788
step 300 , validation loss : 11.3803
step 300 , validation  accuracy 0.866667
step 3

IOError: [Errno 2] No such file or directory: '/ssd/n_vs_ab/type1/1/test_img.npy'

In [ ]:
summary()

In [ ]:
sess.close()